# Imports

In [0]:
import copy
import math
import itertools
import os
import time
from typing import List

import pandas as pd
import json
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import colors
from concurrent.futures import ProcessPoolExecutor, as_completed

# MAIN | VERIFICATION | LOCAL |  REASONING | EXPLORATION
RUN_TYPE = "MAIN"
RUN_LOCAL_ON_EVAL = True
FOCUS_TASK = "0bb8deee.json" # "a87f7484.json"
NUM_TAKS_TO_TRY = 104
VERIFICATION_TYPE = "EXECUTE" # "EXECUTE" | "SEARCH" | "BOTH"

MAX_SEARCH_DEPTH = 4
MAX_COMPLEXITY = 14
MAX_KNOBS = 5  # Useful when MAX_COMPLEXITY is set higher
MAX_INSTANCES_EVALUATED = 50000  # Useful when MAX_SEARCH_DEPTH is high
DEBUG_LEVEL = 0  # 0 is none,  1 is basic, 2 is high, 3 includes program generation

SKIP_LAMBDA_DEPTH = True

# Turning off memo for now, it interferes with some solution
# constructions due to mutable grid and order-dependent methods
# Note: Memo appears to offer around a 40% speedup
ENABLE_MEMO = False  # About a 15-24% speedup @ 10c
ENABLE_EXCLUSIONS = True

# WARNING: This optimization loses important cases!
ENABLE_SYMMETRY_EXCLUSIONS = True  # About a 20% speedup @ 10c
ENABLE_SKIP_LARGE_ITEMS = False

# Debug options for non-main execution
EXECUTION_COUNT_LIMIT = 1000
EXECUTION_COUNT = 0


# Check both the kaggle input directory or the current working directory for local usage
if os.path.exists('/kaggle/input/abstraction-and-reasoning-challenge/'):
    data_path = Path('/kaggle/input/abstraction-and-reasoning-challenge/')
    ENVIRONMENT = "Kaggle"
elif os.path.exists('abstraction-and-reasoning-challenge/'):
    data_path = Path('abstraction-and-reasoning-challenge/')
    ENVIRONMENT = "Local"
else:
    raise Exception("Could not find data.")

# Hard to make this work on Windows?
ENABLE_THREADING = ENVIRONMENT == "Kaggle"

# **Classes and Types**
* Bounds(y,x,Y,X)
* Point(i,j)

In [0]:
class Bounds:
    def __init__(self, y, x, Y, X):
        self.y = y
        self.x = x
        self.Y = Y
        self.X = X

    def __str__(self):
        return "Bounds(" + ", ".join([str(self.y), str(self.x), str(self.Y), str(self.X)]) + ")"

    def __repr__(self):
        return self.__str__()


class Point:
    def __init__(self, i, j):
        self.i = i
        self.j = j

    def __str__(self):
        return '(' + ', '.join([str(self.i), str(self.j)]) + ')'

    def __repr__(self):
        return self.__str__()

# **Core functions**
  * newGrid(size)
  * loop2d(grid, callback)
  * getBoundsOfGrid(grid)
  * map2d(grid, callback)
  * clone2d(grid)
  * joinMultiUsingOr(grids)
  * grabBounds(bounds)
  * grabRegion(grid, i, j, h, w)
  * isInBounds(grid, i, j)
  * splitVertical()

In [0]:
def log(msg):
    if DEBUG_LEVEL > 0:
        print(msg)


def newGrid(height, width):
    if height > 30 or width > 30: return [[0]]
    return [[0 for x in range(width)] for y in range(height)]


def loop2d(grid, callback):
    w = len(grid[0])
    for i in range(len(grid)):
        for j in range(w):
            callback(grid[i][j])


def getBoundsOfGrid(grid):
    height = len(grid)
    width = len(grid[0]) if height > 0 else 0;
    return Bounds(0, 0, height, width)


def map2d(grid, callback):
    w = len(grid[0])
    for i in range(len(grid)):
        for j in range(w):
            grid[i][j] = callback(grid[i][j])
    return grid


def clone2d(grid):
    return [row[:] for row in grid]


def joinMultiUsingOr(grids):
    global GRID
    output = []
    if len(grids) == 0 or len(grids[0]) == 0:
        return [[]]

    lg = len(grids)
    w = len(grids[0][0])
    for i in range(len(grids[0])):
        newRow = []
        for j in range(w):
            value = grids[0][i][j]
            for k in range(1, lg):
                value = grids[k][i][j] if value == 0 else value
            newRow.append(value)
        output.append(newRow)

    GRID = output
    return GRID


def joinMultiUsingXor(twoGrids):
    output = []
    if len(twoGrids) != 2:
        return [[]]

    a = twoGrids[0]
    b = twoGrids[1]
    for i in range(len(a)):
        newRow = []
        for j in range(len(a[0])):
            isXor = (not a[i][j]) != (not b[i][j])
            newRow.append(PRIMARY_COLOR if isXor else BG_COLOR)
        output.append(newRow)

    global GRID
    GRID = output
    return output


def cropToBounds(bounds):
    global GRID
    if bounds is None: return GRID
    GRID = grabRegion(GRID, bounds.y, bounds.x, bounds.Y - bounds.y, bounds.X - bounds.x)
    return GRID


def cropToObject(obj):
    global GRID
    if obj is None: return GRID
    GRID = grabRegion(GRID, obj.y, obj.x, obj.Y - obj.y, obj.X - obj.x)
    return GRID


def grabBounds(grid, bounds):
    if grid is None or bounds is None: return [[0]]
    return grabRegion(grid, bounds.y, bounds.x, bounds.Y - bounds.y, bounds.X - bounds.x)


def grabObject(grid, obj):
    if grid is None or obj is None: return [[0]]
    return grabRegion(grid, obj.y, obj.x, obj.Y - obj.y, obj.X - obj.x)


# Copies the matrix into GRID at i/j
def putMatrix(matrix, i, j):
    w = len(matrix[0])
    for y in range(len(matrix)):
        for x in range(w):
            if isInBounds(GRID, i + y, j + x):
                GRID[i + y][j + x] = matrix[y][x]
    return GRID


def grabRegion(grid, i, j, h, w):
    output = []
    for a in range(i, i + h):
        row = []
        for b in range(j, j + w):
            row.append(grid[a][b] if isInBounds(grid, a, b) else 0)
        output.append(row)
    return output


def isInBounds(grid, i, j):
    return 0 <= i < len(grid) and 0 <= j < len(grid[0])


def splitVertical():
    totalHeight = len(GRID)
    totalWidth = len(GRID[0]) if totalHeight > 0 else 0
    outHeight = math.floor(totalHeight / 2)
    first = grabRegion(GRID, 0, 0, outHeight, totalWidth)
    second = grabRegion(GRID, totalHeight - outHeight, 0, outHeight, totalWidth)
    return [first, second]


# Given some repeating sequence, print it for N steps
def renderSequence(sequence):
    global GRID
    if sequence is None:
        return GRID

    length = ENV["variables"]["OUTPUT_WIDTH"][0] if sequence.direction == "HORIZONTAL" else ENV["variables"]["OUTPUT_HEIGHT"][0]

    grid = newGrid(length, sequence.width)
    for i in range(length):
        grid[i] = sequence.elements[i % sequence.period][:]

    if sequence.direction == 'HORIZONTAL':
        GRID = transpose(grid)
    else:
        GRID = grid

    return GRID

# Color Functions**
  * replaceColor(grid, fromColor, toColor)
  * replaceColorInBounds(fromColor, toColor, bounds)
  * getPrimaryColor(grid)
  * getSecondaryColor(grid)
  * getNthColor(grid, n)

In [0]:


def replaceColor(fromColor, toColor):
    if len(GRID) == 0: return GRID
    return replaceColorInBounds(fromColor, toColor, Bounds(0, 0, len(GRID), len(GRID[0])))


def computeLimitedBounds(grid, bounds):
    y = max(bounds.y, 0)
    Y = min(bounds.Y, len(grid))
    x = max(bounds.x, 0)
    X = min(bounds.X, len(grid[0]))
    return [y, Y, x, X]


def replaceColorInBounds(fromColor, toColor, bounds):
    if len(GRID) == 0 or bounds is None:
        return GRID

    [y, Y, x, X] = computeLimitedBounds(GRID, bounds)
    for i in range(y, Y):
        for j in range(x, X):
            if GRID[i][j] == fromColor:
                GRID[i][j] = toColor
    return GRID


# TODO: Deprecate
def getPrimaryColor(grid):
    return getNthColor(grid, 1)


# TODO: Deprecate
def getSecondaryColor(grid):
    return getNthColor(grid, 2)


# TODO: Deprecate
def getNthColor(grid, n):
    rawCounts = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for row in grid:
        for cell in row:
            rawCounts[cell] = rawCounts[cell] + 1
    sortedCounts = rawCounts[1:]  # skip the background color
    sortedCounts.sort(reverse=True)
    maxVal = sortedCounts[n - 1]
    return rawCounts.index(maxVal)


def getBoundingBoxOfEverything(grid):
    height = len(grid)
    width = len(grid[0]) if height > 0 else 0
    bounds = Bounds(height, width, 0, 0)
    for i in range(height):
        for j in range(width):
            if grid[i][j] != 0:
                bounds = Bounds(min(bounds.y, i), min(bounds.x, j), max(bounds.Y, i + 1), max(bounds.X, j + 1))
    if bounds.y > bounds.Y or bounds.x > bounds.X:
        bounds = Bounds(0, 0, 0, 0)
    return bounds


# TODO: Can I precompute this?
def getBoundingBoxOfColor(color):
    height = len(GRID)
    width = len(GRID[0]) if height > 0 else 0
    bounds = Bounds(height, width, 0, 0)
    for i in range(height):
        for j in range(width):
            if GRID[i][j] == color:
                bounds = Bounds(min(bounds.y, i), min(bounds.x, j), max(bounds.Y, i + 1), max(bounds.X, j + 1))
    if bounds.y > bounds.Y or bounds.x > bounds.X:
        bounds = Bounds(0, 0, 0, 0)
    return bounds


def fillBoundsWithColor(bounds, color):
    if bounds is None: return GRID

    y = max(0, bounds.y)
    x = max(0, bounds.x)
    Y = min(bounds.Y, len(GRID))
    X = min(bounds.X, len(GRID[0]) if len(GRID) > 0 else 0)

    for i in range(y, Y):
        for j in range(x, X):
            GRID[i][j] = color
    return GRID

# Rotation and Flipping functions
  * rotate180()
  * rotate90()
  * flipVertical()
  * flipHorizontal()
  * flipVerticalOverPoint(grid, point)
  * flipHorizontalOverPoint(grid, point)
  * kaleidoscope()
  * kaleidoscopeAroundPoint(grid)
  * joinFourTurns()
  * joinFourTurnsAroundPoint(grid, point)

In [0]:
# unlike rotate90, this currently supports 2d rotation
def rotate180(grid=None):
    global GRID
    if grid is None:
        grid = GRID
    a = clone2d(grid)
    h = len(a)
    if h == 0:
        return [[]]
    w = len(a[0])
    for i in range(h):
        for j in range(w):
            a[i][j] = grid[h - i - 1][w - j - 1]

    # Copy it back
    for i in range(h):
        for j in range(w):
            grid[i][j] = a[i][j]

    return grid


def rotate90(grid=None):
    global GRID
    if grid is None:
        grid = GRID

    n = len(grid)
    if n == 0 or n != len(grid[0]):
        return grid

    for i in range(n // 2):
        for j in range(i, n - i - 1):
            temp = grid[i][j];
            grid[i][j] = grid[n - j - 1][i];
            grid[n - j - 1][i] = grid[n - i - 1][n - j - 1];
            grid[n - i - 1][n - j - 1] = grid[j][n - i - 1];
            grid[j][n - i - 1] = temp;
    return grid


def flipVertical(grid=None):
    global GRID
    if grid is None:
        grid = GRID
    height = len(grid)
    for i in range(height // 2):
        for j in range(len(grid[0])):
            newI = height - 1 - i
            temp = grid[i][j]
            grid[i][j] = grid[newI][j]
            grid[newI][j] = temp
    return grid


def flipHorizontal(grid=None):
    global GRID
    if grid is None:
        grid = GRID
    height = len(grid)
    for i in range(height):
        for j in range(len(grid[0]) // 2):
            newJ = len(grid[0]) - 1 - j
            temp = grid[i][j]
            grid[i][j] = grid[i][newJ]
            grid[i][newJ] = temp
    return grid


def flipVerticalOverPoint(grid, point):
    for i in range(point.i):
        for j in range(len(grid[0])):
            pointDelta = point.i - i
            newI = round(point.i + pointDelta)
            if isInBounds(grid, newI, j):
                temp = grid[i][j]
                grid[i][j] = grid[newI][j]
                grid[newI][j] = temp
    return grid


def flipHorizontalOverPoint(grid, point):
    for i in range(len(grid)):
        for j in range(point.j):
            pointDelta = point.j - j
            newJ = round(point.j + pointDelta)
            if isInBounds(grid, i, newJ):
                temp = grid[i][j]
                grid[i][j] = grid[i][round(newJ)]
                grid[i][round(newJ)] = temp
    return grid


def kaleidoscope():
    global GRID
    GRID = doubleCanvasSize(GRID)
    b = flipVertical(clone2d(GRID))
    c = flipHorizontal(clone2d(GRID))
    d = flipHorizontal(clone2d(b))
    GRID = joinMultiUsingOr([GRID, b, c, d])
    return GRID


def kaleidoscopeAroundPoint(grid, point):
    global GRID
    b = flipVerticalOverPoint(clone2d(grid), point)
    c = flipHorizontalOverPoint(clone2d(grid), point)
    d = flipHorizontalOverPoint(clone2d(b), point)
    GRID = joinMultiUsingOr([grid, b, c, d])
    return GRID


def joinFourTurns():
    global GRID
    GRID = doubleCanvasSize(GRID)
    return joinFourTurnsAroundPoint(GRID, None)


def joinFourTurnsAroundPoint(grid, point):
    global GRID
    a = grid
    b = rotateAroundPoint(clone2d(a), point)
    c = rotateAroundPoint(clone2d(b), point)
    d = rotateAroundPoint(clone2d(c), point)
    GRID = joinMultiUsingOr([a, b, c, d])
    return GRID


def rotateAroundPoint(grid, point):
    if point is None:
        return rotate90(grid)
    # TODO: Implement
    return grid

# **Smore functions**

In [0]:
# Looks for lakes, which are background areas surrounded by non-background
def findLakes(grid, allowTouchingBorder):
    def DFS(i, j, myList):
        if not isInBounds(grid, i, j):
            return allowTouchingBorder
        if visited[i][j]:
            return True
        visited[i][j] = True
        if grid[i][j] == 0:
            myList.append(Point(i, j))
            a = DFS(i - 1, j, myList)
            b = DFS(i + 1, j, myList)
            c = DFS(i, j - 1, myList)
            d = DFS(i, j + 1, myList)
            return a and b and c and d
        return True

    lists = []
    visited = [[False for j in range(len(grid[0]))] for i in range(len(grid))]
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if visited[i][j]:
                continue
            if grid[i][j] == 0:
                myList = []
                isOk = DFS(i, j, myList)
                if isOk:
                    lists.append(myList)
    return list(map(pointsToObjects, lists))


def invert(grid, newColor):
    global GRID
    GRID = grid
    for i in range(len(GRID)):
        for j in range(len(GRID[0])):
            color = GRID[i][j]
            GRID[i][j] = newColor if color == 0 else 0
    return GRID


def splitHorizontal():
    totalHeight = len(GRID)
    totalWidth = len(GRID[0]) if totalHeight > 0 else 0
    outWidth = math.floor(totalWidth / 2)
    first = grabRegion(GRID, 0, 0, totalHeight, outWidth)
    second = grabRegion(GRID, 0, totalWidth - outWidth, totalHeight, outWidth)
    return [first, second]


def joinMultiUsingAnd(grids):
    outputGrid = []
    if len(grids) == 0:
        return [[]]

    for i in range(len(grids[0])):
        newRow = []
        for j in range(len(grids[0][0])):
            value = grids[0][i][j]
            for k in range(1, len(grids)):
                value = grids[k][i][j] if value == grids[k][i][j] else 0
            newRow.append(value)
        outputGrid.append(newRow)

    global GRID
    GRID = outputGrid
    return outputGrid


COLORS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


def getAllColorBounds(grid):
    allBounds = []
    for c in COLORS:
        if c != 0:
            bounds = getBoundingBoxOfColor(c)
            if bounds.X > bounds.x and bounds.Y > bounds.y:
                allBounds.append(bounds)
    return allBounds


def getBoundsSize(bounds):
    return (bounds.Y - bounds.y) * (bounds.X - bounds.x)


# Dot is a Object
class Dot:
    def __init__(self, i, j, color):
        self.i = i
        self.j = j
        self.color = color

    def __str__(self):
        return "(" + str(self.i) + ", " + str(self.j) + ")"

    def __repr__(self):
        return "Dot" + self.__str__()


def drawDotBorder(dot, color):
    for i in range (-1, 2):
        for j in range(-1, 2):
            if isInBounds(GRID, dot.i + i, dot.j + j) and (i != 0 or j != 0):
                GRID[dot.i + i][dot.j + j] = color
    return GRID


# Mirrors the grid over the bottom, keeping the bottom 'offset' 0 or 1 px
def mirrorVertical(offset):
    global GRID
    height = len(GRID)

    if height == 0 or height >= 15:
        return GRID

    new_height = height * 2 - offset

    width = len(GRID[0])
    new_grid = newGrid(new_height, width)

    if len(new_grid) != new_height:
        return GRID # Maybe we tried to scale too large


    for i in range(len(GRID)):
        for j in range(width):
            new_grid[i][j] = GRID[i][j]
            new_grid[new_height - i - 1][j] = GRID[i][j]

    GRID = new_grid
    return GRID



def drawBorder(bounds, color):
    if bounds is None: return GRID

    x = bounds.x - 1
    y = bounds.y - 1
    X = bounds.X
    Y = bounds.Y

    drawCardinalLine(Point(y, x), Point(y, X), color, False)
    drawCardinalLine(Point(y, X), Point(Y, X), color, False)
    drawCardinalLine(Point(Y, X), Point(Y, x), color, False)
    drawCardinalLine(Point(Y, x), Point(y, x), color, False)

    return GRID


def fractal():
    global GRID
    if len(GRID) == 0 or len(GRID) > 5 or len(GRID) != len(GRID[0]):
        return GRID

    old_size = len(GRID)
    size = old_size * old_size
    new_grid = newGrid(size, size)
    for i in range(old_size):
        for j in range(old_size):
            for x in range(old_size):
                for y in range(old_size):
                    if GRID[i][j]:
                        new_grid[i * old_size + y][j * old_size + x] = GRID[y][x]

    GRID = new_grid
    return GRID


def filterToLines(objects):
    lines = []
    for ob in objects:
        if (ob.height == 1 and ob.width > 1) or (ob.width == 1 and ob.height > 1):
            lines.append(ob)

    return lines


def clear():
    global GRID
    if len(GRID) > 0:
        GRID = newGrid(len(GRID), len(GRID[0]))
    return GRID


# Draws the object to the grid, ex: it might have been cleared before
def putObject(obj):
    if obj is None or obj.grid is None or len(obj.grid) == 0: return GRID

    for i in range(len(obj.grid)):
        for j in range(len(obj.grid[0])):
            if isInBounds(GRID, i + obj.y, j + obj.x):
                if obj.grid[i][j] == obj.color:
                    GRID[i + obj.y][j + obj.x] = obj.grid[i][j]
    return GRID

def filterDotsByColor(dots, color):
    return list(filter(lambda dot: dot.color == color, dots))


def hasNoNeighbors(input_grid, i, j):
    up = not isInBounds(input_grid, i - 1, j) or input_grid[i - 1][j] == 0
    down = not isInBounds(input_grid, i, j + 1) or input_grid[i][j + 1] == 0
    left = not isInBounds(input_grid, i + 1, j) or input_grid[i + 1][j] == 0
    right = not isInBounds(input_grid, i, j - 1) or input_grid[i][j - 1] == 0

    return up and down and left and right


def findDots(input_grid):
    dots = []
    for i in range(len(input_grid)):
        for j in range(len(input_grid[0])):
            if input_grid[i][j] != 0:  # BG_COLOR
                if hasNoNeighbors(input_grid, i, j):
                    dots.append(Dot(i, j, input_grid[i][j]))

    return dots


def hasNoMatchingNeighbors(input_grid, i, j):
    color = input_grid[i][j]
    up = not isInBounds(input_grid, i - 1, j) or input_grid[i - 1][j] != color
    down = not isInBounds(input_grid, i, j + 1) or input_grid[i][j + 1] != color
    left = not isInBounds(input_grid, i + 1, j) or input_grid[i + 1][j] != color
    right = not isInBounds(input_grid, i, j - 1) or input_grid[i][j - 1] != color

    return up and down and left and right


# Finds "colordots" which are pixels that have no adjacent
# pixels of the same color
def findCDots(input_grid):
    dots = []
    for i in range(len(input_grid)):
        for j in range(len(input_grid[0])):
            if input_grid[i][j] != 0:  # BG_COLOR
                if hasNoMatchingNeighbors(input_grid, i, j):
                    dots.append(Dot(i, j, input_grid[i][j]))

    return dots


def forEachBounds(boundsList, callback):
    for bounds in boundsList:
        callback(bounds)
    return GRID


def doBoth(eh, beh):
    # Don't actually do anything with the inputs
    return beh


def forEachDot(dots, callback):
    for dot in dots:
        callback(dot)
    return GRID


def forEachObject(objects, callback):
    for obj in objects:
        callback(obj)
    return GRID


def forEachInputColor(callback):
    for c in INPUT_COLORS:
        if c != 0:
            callback(c)
    return GRID


def getColoredObject(color):
    objs = COLORED_OBJECTS[color]
    if len(objs) > 0:
        return objs[0]

    return None


def getColoredObjects(color):
    return COLORED_OBJECTS[color]


def drawLine(dot, dy, dx, bidi=False):
    return drawColoredLine(dot, dy, dx, dot.color, bidi)


def drawColoredLine(start, dy, dx, color, bidi=False, stopOnImpact=False):
    if dy == 0 and dx == 0:
        return GRID

    i = start.i + dy
    j = start.j + dx

    while isInBounds(GRID, i, j):
        if stopOnImpact and GRID[i][j] != 0:
            break
        GRID[i][j] = color
        i = i + dy
        j = j + dx

    if bidi:
        drawColoredLine(start, dy * -1, dx * -1, color, bidi=False)

    return GRID

def drawLineFromObject(startObject, dy, dx):
    if startObject is None: return GRID
    return drawColoredLineFromObject(startObject, dy, dx, startObject.color)


def getLineStartingPointOnObject(obj, dy, dx):
    if dy == -1 and dx == -1:
        return Dot(obj.y, obj.x, obj.color)
    if dy == 1 and dx == -1:
        return Dot(obj.Y - 1, obj.x, obj.color)
    if dy == -1 and dx == 1:
        return Dot(obj.y, obj.X - 1, obj.color)
    if dy == 1 and dx == 1:
        return Dot(obj.Y - 1, obj.X - 1, obj.color)

    if dy == 0 and dx == -1:
        return Dot((obj.y + obj.Y) // 2, obj.x, obj.color)
    if dy == 0 and dx == 1:
        return Dot((obj.y + obj.Y) // 2, obj.X - 1, obj.color)
    if dy == -1 and dx == 0:
        return Dot(obj.y, (obj.x + obj.X) // 2, obj.color)
    if dy == 1 and dx == 0:
        return Dot(obj.Y - 1, obj.x, obj.color)

    return Dot((obj.y + obj.Y) // 2, (obj.x + obj.X) // 2, obj.color)


def drawColoredLineFromObject(startObject, dy, dx, color):
    if startObject is None:
        return GRID

    start = getLineStartingPointOnObject(startObject, dy, dx)
    return drawColoredLine(start, dy, dx, color)



def drawLineToObject(dot, object):
    if dot is None or object is None: return GRID
    dx = 0
    dy = 0

    if dot.i >= object.Y:
        dy = -1
    if dot.i < object.y:
        dy = 1
    if dot.j >= object.X:
        dx = -1
    if dot.j < object.x:
        dx = 1
    drawColoredLine(dot, dy, dx, dot.color, stopOnImpact=True)

    return GRID


def moveDotToObject(dot, object):
    if dot is None or object is None: return GRID
    if not isInBounds(GRID, dot.i, dot.j): return GRID

    dy = 0
    dx = 0
    if dot.i >= object.Y:
        dy = -1
    if dot.i < object.y:
        dy = 1
    if dot.j >= object.X:
        dx = -1
    if dot.j < object.x:
        dx = 1

    y = dot.i
    x = dot.j
    while isInBounds(GRID, y + dy, x + dx):
        if isInsideObject(Point(y + dy, x + dx), object):
            GRID[y][x] = dot.color
            GRID[dot.i][dot.j] = 0
            break;
        y += dy
        x += dx

    return GRID


def isInsideObject(point, object):
    return object.x <= point.j < object.X and object.y <= point.i < object.Y;


def drawLineAwayFromObject(dot, object):
    if dot is None or object is None: return GRID
    dx = 0
    dy = 0

    if dot.i >= object.Y:
        dy = 1
    if dot.i < object.y:
        dy = -1
    if dot.j >= object.X:
        dx = 1
    if dot.j < object.x:
        dx = -1
    drawColoredLine(dot, dy, dx, dot.color)

    return GRID


def drawCardinalLine(fromPoint, toPoint, color, onlyWriteOnBg):
    if fromPoint.i == toPoint.i:
        if fromPoint.j < toPoint.j:
            first = fromPoint
            second = toPoint
        else:
            first = toPoint
            second = fromPoint

        i = fromPoint.i
        for j in range(first.j, second.j + 1):
            if isInBounds(GRID, i, j):
                if not onlyWriteOnBg or GRID[i][j] == 0:
                    GRID[i][j] = color
    else:
        if fromPoint.i < toPoint.i:
            first = fromPoint
            second = toPoint
        else:
            first = toPoint
            second = fromPoint

        j = fromPoint.j
        for i in range(first.i, second.i + 1):
            if isInBounds(GRID, i, j):
                if not onlyWriteOnBg or GRID[i][j] == 0:
                    GRID[i][j] = color


# Writes on an assumed grid
def connectColinearPoints(points, color):
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            a = points[i]
            b = points[j]
            if a.i == b.i or a.j == b.j:
                drawCardinalLine(a, b, color, True)
    return GRID


# if v is 0, take the full size
# if v is 1 take the top half, vertically
# if v is 2 take the bottom half, vertically
def grabPart(obj, vert, hori):
    grid = grabObject(GRID, obj)

    grid_height = len(grid)
    if len(grid) == 0:
        return grid

    grid_width = len(grid[0])

    out_height = grid_height if vert == 0 else grid_height // 2
    out_width = grid_width if hori == 0 else grid_height // 2
    out_start_y = 0 if vert != 2 else grid_height - out_height
    out_start_x = 0 if hori != 2 else grid_width - out_width

    out_grid = newGrid(out_height, out_width)
    if (len(out_grid) != out_height):
        return [[0]]

    for i in range(out_height):
        for j in range(out_width):
            if isInBounds(grid, out_start_y + i, out_start_x + j):
                out_grid[i][j] = grid[out_start_y + i][out_start_x + j]

    return out_grid


def filterOutDots(boundsList):
    return list(filter(lambda bounds: getBoundsSize(bounds) > 1, boundsList))


def sortLargestFirst(boundsList):
    return sorted(boundsList, key=getBoundsSize, reverse=True)


def sortSmallestFirst(boundsList):
    return sorted(boundsList, key=getBoundsSize)


# Returns only the smallest bounds of the boundsList
def findSmallest(boundsList):
    if len(boundsList) == 0:
        return None
    return sortSmallestFirst(boundsList)[0]


def findLargest(boundsList):
    if len(boundsList) == 0:
        return None

    return sortLargestFirst(boundsList)[0]


def area(obj):
    return (obj.Y - obj.y) * (obj.X - obj.x)


# Returns a grid that is the result of tiling an object
# (grabbed from GRID) ny times vertically and nx times horizontally
def repeatTile(obj, ny, nx):
    global GRID
    tile = grabObject(GRID, obj)
    if len(tile) == 0:
        return GRID

    tile_height = len(tile)
    tile_width = len(tile[0])

    GRID = newGrid(ny * tile_height, nx * tile_width)
    for i in range(ny):
        for j in range(nx):
            putMatrix(tile, i * tile_height, j * tile_width)

    return GRID


# Returns the number of different colors inside the bounds of an object
def countColors(obj):
    return len(enumerateColors(grabObject(GRID, obj)))


# Returns the full identity of an object
def identity(obj):
    return obj.fullHash


def bwIdentity(obj):
    return obj.bwHash


# Returns a list of objects derived from slicing the input
def slice(h, w, pad):
    grid_height = len(GRID)
    grid_width = len(GRID[0])
    bounds = []
    for I in range(grid_height // (h + pad)):
        for J in range(grid_width // (w + pad)):
            y = I * (h + pad)
            x = J * (w + pad)
            bounds.append(Bounds(y, x, y + h, x + w))

    return objectifyBoundsList(bounds, GRID)


def findUnique(objects, valueCallback):
    tuples = []
    for obj in objects:
        key = valueCallback(obj);
        tuples.append([str(key), obj])
    counts = dict()
    for pair in tuples:
        if pair[0] in counts:
            counts[pair[0]] += 1
        else:
            counts[pair[0]] = 1
    for pair in tuples:
        if counts[pair[0]] == 1:
            return pair[1]

    # TODO: This should return nothing and end the call qequence probably
    if len(objects) > 0:
        return objects[0]
    else:
        return Object(Bounds(0, 0, 0, 0))

In [0]:
def findBiggestBoxes(mat):
    boxes = []
    for i in range(len(mat)):
        for j in range(len(mat[i])):
            color = mat[i][j]
            if color == 0:
                continue
            maxY = i
            maxX = j
            for y in range(i, len(mat)):
                if mat[y][j] != color:
                    break
                maxY = y
                for x in range(j, len(mat[0])):
                    if mat[y][x] != color:
                        break
                    maxX = x
            boxes.append(Bounds(i, j, maxY + 1, maxX + 1))

    sortedBoxes = sorted(boxes, key=lambda b: (b.X - b.x) * (b.Y - b.y), reverse=True)
    visited = [[0 for j in mat[0]] for i in mat]
    foundBoxes = []
    for box in sortedBoxes:
        if isBoxUnvisited(box, visited):
            foundBoxes.append(box)
            visitBox(box, visited, len(foundBoxes))

    return objectifyBoundsList(foundBoxes, mat)


def isBoxUnvisited(box, visited):
    for i in range(box.y, box.Y):
        for j in range(box.x, box.X):
            if visited[i][j] != 0:
                return False
    return True


def visitBox(box, visited, value):
    for i in range(box.y, box.Y):
        for j in range(box.x, box.X):
            visited[i][j] = value

# Translation
* translateDot

In [0]:
def translateDot(dot, i, j):
    GRID[dot.i][dot.j] = 0
    GRID[dot.i + i][dot.j + j] = dot.color
    return GRID


# TODO: object should not move the entire bounds, but rather the pixels?
def translateObject(object, di, dj):
    if object is None: return GRID
    source = clone2d(GRID)

    for i in range(object.y, object.Y):
        for j in range(object.x, object.X):
            if isInBounds(GRID, i, j):
                GRID[i][j] = 0

    for i in range(object.y, object.Y):
        for j in range(object.x, object.X):
            if isInBounds(GRID, i + di, j + dj) and isInBounds(GRID, i, j):
                GRID[i + di][j + dj] = source[i][j]

    return GRID

def moveObjectToAlignWithObject(a, b, axis):
    # TODO: implement
    return GRID

# Moves a until it touches b
def moveObjectToTouchObject(a, b):
    if a is None or b is None:
        return GRID

    ac = getCenterPointOf(a, False)
    bc = getCenterPointOf(b, False)
    di = bc.i - ac.i
    dj = bc.j - ac.j

    # TODO: Compute x or y as mode of translation
    if abs(di) > abs(dj):
        if di > 0:
            gap = b.y - a.Y
            translateObject(a, gap, 0)
        else:
            gap = b.Y - a.y
            translateObject(a, gap, 0)
    else:
        if dj > 0:
            gap = b.x - a.X
            translateObject(a, 0, gap)
        else:
            gap = b.X - a.x
            translateObject(a, 0, gap)

    return GRID


def applyColorSequence(objectList, colorList, callback):
    if len(objectList) == 0 or len(colorList) == 0:
        return GRID

    for i in range(len(objectList)):
        color = colorList[i % len(colorList)]
        callback(objectList[i], color)

    return GRID

# Bounds and Selection
  * changeBounds(bounds, dy, dx)
  * shrinkBounds(bounds)
  * growBounds(bounds)
  * gridBoundsIter(boundsList, callback)
  * findIslands
  * getLargestSquareBounds
  * getCenterPointOf

In [0]:
def changeBounds(bounds, dy, dx):
    if bounds == None: return None
    return Bounds(bounds.y - dy, bounds.x - dx, bounds.Y + dy, bounds.X + dx)


def shrinkBounds(bounds):
    return changeBounds(bounds, -1, -1)


def growBounds(bounds):
    return changeBounds(bounds, 1, 1)


# Takes a list of points and returns their bounding box
def pointsToObjects(points):
    bounds = Object(Bounds(points[0].i, points[0].j, 0, 0))

    for point in points:
        bounds = Object(Bounds(min(bounds.y, point.i), min(bounds.x, point.j), max(bounds.Y, point.i + 1),
                        max(bounds.X, point.j + 1)))
    return bounds


# Returns a list of all non-bg colors in an image
def enumerateColors(input_grid):
    counts = findColorDistribution(input_grid)

    output = []
    for i in range(0, 10):
        if counts[i] > 0:
            output.append(i)

    return output


def findColorDistribution(input_grid):
    counts = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for row in input_grid:
        for cell in row:
            counts[cell] = counts[cell] + 1
    return counts


def singleColorFilter(input_grid, color):
    return map2d(input_grid, lambda cell: cell if cell == color else 0)


def getPrecomputedMonoObjects(colored_objects):
    largest_size = 0;
    largest = Bounds(0, 0, 1, 1)
    smallest_size = 9001
    smallest = Bounds(0, 0, 1, 1)

    monoObjects = []
    for color in range(1, len(colored_objects)):
        boundsList = colored_objects[color]
        for bounds in boundsList:
            newSize = getBoundsSize(bounds)
            if newSize >= largest_size:
                largest_size = newSize
                largest = bounds
            if newSize <= smallest_size:
                smallest_size = newSize
                smallest = bounds
            if newSize > 0:
                monoObjects.append(Object(bounds, GRID, color))

    return {
        "MONO_OBJECTS": monoObjects,
        "LARGEST_MONO_OBJECT": Object(largest, GRID),
        "SMALLEST_MONO_OBJECT": Object(smallest, GRID)
    }


def findColoredObjects(input_grid):
    result = []

    allColors = enumerateColors(input_grid)
    for color in range(10):
        if color in allColors:
            boundsList = findIslands(singleColorFilter(clone2d(input_grid), color), True)
            objectList = objectifyBoundsList(boundsList, input_grid)
            result.append(objectList)
        else:
            result.append([])

    return result

def findIslands(grid, allowDiagonals=False):
    def DFS(i, j, myList):
        nonlocal visited
        if not isInBounds(grid, i, j) or visited[i][j]:
            return
        visited[i][j] = True
        if grid[i][j] != 0:
            myList.append(Point(i, j))
            if allowDiagonals:
                DFS(i - 1, j - 1, myList)
                DFS(i + 1, j - 1, myList)
                DFS(i - 1, j + 1, myList)
                DFS(i + 1, j + 1, myList)
            DFS(i - 1, j, myList)
            DFS(i + 1, j, myList)
            DFS(i, j - 1, myList)
            DFS(i, j + 1, myList)

    lists = []
    visited = [[0 for i in range(len(grid[0]))] for j in range(len(grid))]
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if visited[i][j]:
                continue
            if grid[i][j] != 0:
                myList = []
                DFS(i, j, myList)
                lists.append(myList)
    return objectifyBoundsList(list(map(pointsToObjects, lists)), grid)


def getLargestSquareBounds(grid):
    smallEdge = min(len(grid), len(grid[0])) - 1
    return Bounds(0, 0, smallEdge, smallEdge)


def getCenterPointOf(bounds, exact):
    if bounds is None:
        return None

    if exact:
        return Point((bounds.Y + bounds.y - 1) / 2, (bounds.X + bounds.x - 1) / 2)
    else:
        return Point((bounds.Y + bounds.y) // 2, (bounds.X + bounds.x) // 2)


def doubleCanvasSize(grid):
    outputGrid = []

    # You never need to scale up a grid that could exceed max size
    if len(grid) == 0 or len(grid) > 15 or len(grid[0]) > 15:
        return [[0]]

    for i in range(len(grid) * 2):
        row = []
        for j in range(len(grid[0]) * 2):
            if isInBounds(grid, i, j):
                row.append(grid[i][j])
            else:
                row.append(0)
        outputGrid.append(row)
    return outputGrid


def scaleUp(factor):
    global GRID
    outputGrid = []

    # You never need to scale up a grid that could exceed max size
    if GRID is None or len(GRID) == 0 or len(GRID) > 15 or len(GRID[0]) > 15:
        return [[0]]

    for i in range(len(GRID)):
        newRows = [[] for f in range(factor)]
        for j in range(len(GRID[0])):
            for index in range(factor):
                newRows[index].extend([GRID[i][j] for f in range(factor)])
        outputGrid.extend(newRows)

    GRID = outputGrid
    return GRID


# Mutation function operating on GRID which draws an Object at a Point
def drawObjectCenteredOnPoint(obj, point):
    if obj is None or point is None: return GRID
    y = round(point.i) - obj.height // 2;
    x = round(point.j) - obj.width // 2;
    # print("Drawing object " + str(obj.height) + "," + str(obj.width) + " at " + str(y) + "," + str(x))
    for i in range(obj.height):
        for j in range(obj.width):
            if isInBounds(GRID, y + i, x + j) and isInBounds(obj.grid, i, j):
                GRID[y + i][x + j] = obj.grid[i][j] if obj.grid[i][j] else 0
    return GRID


class Sequence():
    def __init__(self, elements, period, direction):
        self.elements = elements
        self.period = period
        self.width = len(elements[0]) if len(elements) > 0 else 0
        # Information about the sequence when it was discovered
        self.direction = direction # "VERTICAL" | "HORIZONTAL"
        self.initialLength = len(elements)


class Object(Bounds):
    def __init__(self, bounds, grid=None, color=None):
        super().__init__(bounds.y, bounds.x, bounds.Y, bounds.X)
        self.height = bounds.Y - bounds.y
        self.width = bounds.X - bounds.x
        # Plus add some metadata
        self.fullHash = -1
        self.bwHash = -1
        self.color = color or -1
        self.grid = [[0]]
        if grid:
            self.grid = grabBounds(grid, bounds)
            self.fullHash = hashMatrix(self.grid, isGrayscale=False)
            self.bwHash = hashMatrix(self.grid, isGrayscale=True)
            self.color = color or getPrimaryColor(self.grid)


# Change a bounds list into metadata
def objectifyBoundsList(boundsList, grid):
    objectList = []
    for bounds in boundsList:
        if bounds is not None:
            obj = Object(bounds, grid)
            obj.grid = grabBounds(grid, obj)
            objectList.append(obj)
    return objectList


def hashMatrix(mat, isGrayscale):
    values = []
    for r in range(4):
        val = 0
        c = 1
        for i in range(len(mat)):
            for j in range(len(mat[0])):
                if mat[i][j] > 0:
                    if isGrayscale:
                        val += c
                        c *= 2
                    else:
                        val += mat[i][j] * c
                        c *= 10
        values.append(val)
    return max(values)


# Gives a score ranging from 0 to 1 as the percentage of pixels in the
# two grids that are exactly alike
# NOTE: This currently ignores background!
def gradeGridSimilarity(a, b):
    if len(a) == 0 or len(b) == 0:
        return 0

    y = 0
    x = 0
    Y = min(len(a), len(b))
    X = min(len(a[0]), len(b[0]))
    px = (Y - y) * (X - x)
    score = 0
    for i in range(y, Y):
        for j in range(x, X):
            if a[i][j] == b[i][j] and a[i][j] != 0:
                score += 1 / px

    return score



# TODO: Generalize this process to scoring all potential functions?
# Looks at the grid itself for symmetry (not looking at pairs)
# TODO: Need to focus on the delta between pairs, not just within
def findSymmetryStats(grid):
    # Check if there is rotational symmetry
    grid90 = rotate90(clone2d(grid))
    rot90_score = gradeGridSimilarity(grid, grid90)
    grid180 = rotate180(clone2d(grid))
    rot180_score = gradeGridSimilarity(grid, grid180)

    # Check if there is horizontal symmetry
    grid_h = flipHorizontal(clone2d(grid))
    grid_h_score = gradeGridSimilarity(grid, grid_h)

    # Check if there is vertical symmetry
    grid_v = flipHorizontal(clone2d(grid))
    grid_v_score = gradeGridSimilarity(grid, grid_v)

    return {
        "rotational": (rot90_score + rot180_score) / 2,
        "bilateral": (grid_h_score + grid_v_score) / 2
    }

def makeGridStats(grid):
    islandBounds = findIslands(grid)
    islandObjects = objectifyBoundsList(islandBounds, grid)
    bounds_by_color = findColoredObjects(grid)
    coloredBoundsList = itertools.chain.from_iterable(bounds_by_color)
    coloredObjects = objectifyBoundsList(coloredBoundsList, grid)
    symmetryData = findSymmetryStats(grid)


    return {
        "height": len(grid),
        "width": len(grid[0]),
        "islands": islandBounds,
        "lakes": findLakes(grid, False),
        "dots": findDots(grid),
        "cdots": findCDots(grid),
        "colors": enumerateColors(grid),
        "color_distribution": findColorDistribution(grid),
        "bounds_by_color": bounds_by_color,
        "island_objects": islandObjects,
        "colored_objects": coloredObjects,
        "symmetryData": symmetryData
    }


def computeSizeComparison(a, b):
    aw = a["width"]
    bw = b["width"]
    ah = a["height"]
    bh = b["height"]
    xScale = bw / aw
    yScale = bh / ah
    baseData = {
        "didSizeChange": xScale != 1 or yScale != 1,
        "hasWidthChanges": xScale != 1,
        "hasHeightChanges": yScale != 1,
        "yScale": xScale,
        "xScale": yScale,
        "outputHeight": b["height"],
        "outputWidth": b["width"],
        "isLong": aw > 1.5 * ah,
        "isTall": ah > 1.5 * aw
    }
    return baseData


def computeObjectComparison(a, b):
    islandCountDiff = len(b["islands"]) - len(a["islands"])
    lakesCountDiff = len(b["lakes"]) - len(a["lakes"])
    dotsCountDiff = len(b["dots"]) - len(a["dots"])

    baseData = {
        "islandCountDiff": islandCountDiff,
        "lakesCountDiff": lakesCountDiff,
        "dotsCountDiff": dotsCountDiff,

    }
    return baseData


def computeColorComparison(a, b):
    didColorsChange = b["color_distribution"] != a["color_distribution"]
    colorDeltaDistribution = list(map(lambda f, g: g - f, a["color_distribution"], b["color_distribution"]))
    removedColors = list(filter(lambda c: c in a["colors"] and c not in b["colors"], range(10)))
    addedColors = list(filter(lambda c: c not in a["colors"] and c in b["colors"], range(10)))

    baseData = {
        "didColorsChange": didColorsChange,
        "colorDeltaDistribution": colorDeltaDistribution,
        "removedColors": removedColors,  # Colors which existed in 'a' but do not in 'b'
        "addedColors": addedColors,  # Colors which did not exist in 'a' but do in 'b'
    }
    return baseData


def comparePair(input_grid, output_grid):
    a = makeGridStats(input_grid)
    b = makeGridStats(output_grid)

    return {
        "input": a,
        "output": b,
        "size": computeSizeComparison(a, b),
        "object": computeObjectComparison(a, b),
        "color": computeColorComparison(a, b)
    }


def intersectionOfListProps(input_list, key_a, key_b):
    base_set = set(input_list[0][key_a][key_b])
    for idx in range(1, len(input_list)):
        base_set = base_set & set(input_list[idx][key_a][key_b])
    return list(base_set)


def deriveConclusions(comparison_list, shouldPrint):
    # Verbs: "color", "size", ?
    verbs = []  # A non-scientific set of verbs. Will be used to filter methods
    exclusions = []
    conclusions = {
        "verbs": [],
        "exclusions": [],
        "variables": [],

        # NOT USED YET
        "types": [],
        "operators": []
    }

    comparison_data = comparison_list[0]
    cd = comparison_data
    sizeData = cd["size"]
    objectData = cd["object"]
    colorData = cd["color"]
    if sizeData["didSizeChange"]:
        if shouldPrint: print("size changed. Including size operations")
        verbs.append("size")

        if sizeData["isTall"] and sizeData["yScale"] > 1:
            if shouldPrint: print("Got even taller. Vertical sequence?")
        if sizeData["isLong"] and sizeData["xScale"] > 1:
            if shouldPrint: print("Got even longer. Horizontal sequence?")
    else:
        exclusions.append("size")
        if shouldPrint: print("There were no changes in size. Ignoring sizing methods")

    outputColors = intersectionOfListProps(comparison_list, "output", "colors")
    outputColors.append("PRIMARY_COLOR")
    outputColors.append("SECONDARY_COLOR")

    inputColors = intersectionOfListProps(comparison_list, "input", "colors")
    global INPUT_COLORS
    INPUT_COLORS = inputColors[:]
    inputColors.append("PRIMARY_COLOR")
    inputColors.append("SECONDARY_COLOR")
    conclusions["variables"].append(Type("OUTPUT_COLOR", outputColors, "color"))
    conclusions["variables"].append(Type("INPUT_COLOR", inputColors, "color"))
    conclusions["variables"].append(Type("OUTPUT_HEIGHT", [sizeData["outputHeight"]]))
    conclusions["variables"].append(Type("OUTPUT_WIDTH", [sizeData["outputWidth"]]))

    if colorData["didColorsChange"]:
        verbs.append("color")
        if shouldPrint: print("There was at least some change in the color distribution. Using color-related methods")
        if shouldPrint: print("A major false positive is for single-color images with shape changes")

        if len(colorData["addedColors"]) > 0:
            if shouldPrint: print("New colors were added: " + str(colorData["addedColors"]) + ".")
            # conclusions["variables"].append(Type("OUTPUT_ONLY_COLOR", colorData["addedColors"], "OUTPUT_COLOR"))

        if len(colorData["removedColors"]) > 0:
            if shouldPrint: print("Some colors were removed: " + str(colorData["removedColors"]) + ".")
            # conclusions["variables"].append(Type("INPUT_ONLY_COLOR", colorData["removedColors"], "INPUT_COLOR"))
    else:
        if shouldPrint: print("There was no change in the color distribution. Ignoring color mutation methods.")

    input_data = comparison_data["input"]
    if len(input_data["dots"]) > 30 or len(input_data["cdots"]) > 30:
        exclusions.append("too_many_dots")

    # Handle Symmetry
    if shouldPrint:
        print("Symmetry Summary (2 Rotational then 2 Bilateral):")
        print("  " + str(cd["input"]["symmetryData"]["rotational"]))
        print("  " + str(cd["output"]["symmetryData"]["rotational"]))
        print("  " + str(cd["input"]["symmetryData"]["bilateral"]))
        print("  " + str(cd["output"]["symmetryData"]["bilateral"]))
    if cd["input"]["symmetryData"]["rotational"] < 0.5 and cd["output"]["symmetryData"]["rotational"] < 0.5:
        if shouldPrint: print("Input and output appear to lack rotational symmetry")
        exclusions.append("rotation")

    if cd["input"]["symmetryData"]["bilateral"] < 0.5 and cd["output"]["symmetryData"]["bilateral"] < 0.5:
        if shouldPrint: print("Input and output appear to lack bilateral symmetry")
        exclusions.append("bilateral")

    # Handle Objects
    conclusions["verbs"] = verbs
    conclusions["exclusions"] = exclusions
    return conclusions


# Represents an application of a single basic template
# (Ex: an operation with a small set of parameters)
class Application:
    def __init__(self, template):
        self.operation = template


def buildAllTemplateInstances(template):
    instances = []
    if len(template.knobDict) > 0:
        all_options = cartesianProductDict(template.knobDict)
        for selection in all_options:
            instance = makeSolutionInstance(template.templateString, selection)
            instances.append(instance)
    else:
        instance = makeSolutionInstance(template.templateString, {})
        instances.append(instance)

    return instances


# List a set of independent operations to try, given a task+conclusions
def buildInstancesToTry(task, conclusions):
    programs = generateAllProgramTreesUpToDepthN(2, 20, Constraints(returns="grid"), [])
    # An important step to ensure that no two node instances are identical objects
    programs = list(map(copy.deepcopy, programs))
    if DEBUG_LEVEL >= 1:
        for p in programs:
            print(p)
    if DEBUG_LEVEL >= 2:
        print("There are " + str(len(programs)) + " programs to make")

    templates = buildTemplatesAndMemo(programs)
    if DEBUG_LEVEL >= 1:
        for t in templates:
            print(t.templateString)

    if DEBUG_LEVEL >= 2:
        print("There are " + str(len(templates)) + " templates to try")

    instances = []
    for template in templates:
        templateInstances = buildAllTemplateInstances(template)
        if DEBUG_LEVEL >= 2:
            print("Made " + str(len(templateInstances)) + " more instances")
        instances.extend(templateInstances)

    return instances


def gradeResultAgainstActual(result, actual):
    # TODO: Is there a meaninful grading scheme when these sizes differ?
    if len(result) != len(actual) or len(result[0]) != len(actual[0]):
        return 0

    matchingCount = 0
    pixelCount = len(actual) * len(actual[0])
    for i in range(len(actual)):
        for j in range(len(actual[0])):
            matchingCount += 1 if result[i][j] == actual[i][j] else 0
    score = matchingCount / pixelCount
    return score


# Applies the operation to the task
def tryAndScoreInstance(task, instance, conclusions):
    scores = []
    training_pairs = task["train"]
    for i in range(len(training_pairs)):
        pair = training_pairs[i]
        setPrecomputedValues(conclusions["precomps"][i])
        result = executeSolution(instance, pair["input"])
        if DEBUG_LEVEL >= 2:
            print("Result: " + str(result))
            print("Expect: " + str(pair["output"]))
        score = gradeResultAgainstActual(result, pair["output"])
        scores.append(score)

    return min(scores)


def tryStuffAndScoreIt(task):
    conclusions = setTaskwideValues(task)

    instances = buildInstancesToTry(task, conclusions)

    if DEBUG_LEVEL >= 2:
        print("There are " + str(len(instances)) + " instances to try")

    for t in instances:
        score = tryAndScoreInstance(task, t, conclusions)
        if score == 1:
            print("Solved!")
            return True


def runExploration():
    task_path = training_path
    tasks = sorted(os.listdir(task_path))

    attempts = 0
    solves = 0
    start_time = time.time()
    for i in range(len(tasks)):
        if attempts >= NUM_TAKS_TO_TRY:
            break

        if FOCUS_TASK is not None and tasks[i] != FOCUS_TASK:
            continue

        step_start = time.time()

        with open(str(task_path / tasks[i]), 'r') as f:
            task = json.load(f)

        print("Attempting " + tasks[i])
        # plot_task(task)

        didSolve = tryStuffAndScoreIt(task)

        attempts = attempts + 1
        if didSolve:
            solves = solves + 1
            print("Solved " + tasks[i] + ". now " + str(solves) + "/" + str(attempts))
        else:
            print("Failed " + tasks[i] + ". now " + str(solves) + "/" + str(attempts))
        print("Completed in " + str(time.time() - step_start) + "s")
        print("")

        end_time = time.time()
        print("Time taken: " + str(end_time - start_time))

        print("Attempted " + str(attempts) + " problems:")
        print("Solved " + str(solves) + "/" + str(attempts) + ".")


def reasonAboutTask(task, shouldPrint=False):
    # plot_full_task(task)

    comparison_list = []

    for pair in task["train"]:
        comparison_data = comparePair(pair["input"], pair["output"])
        comparison_list.append(comparison_data)

    shouldPrint = shouldPrint or DEBUG_LEVEL >= 2
    conclusions = deriveConclusions(comparison_list, shouldPrint)

    return conclusions

**Functional tests**
#
#  The next block contains a small set of unit tests which I wrote while ensuring the functions each work like
#  expected. The functions themselves may need to be tweaked, so maintaining this list is important to ensure that
#  solutions continue working as expected.

In [0]:
def expect(actual, expected):
    if actual != expected:
        print("Expected actual (first) to equal expected (second)")
        print("    " + str(actual))
        print("    " + str(expected))
        return False
    return True


def runFunctionalTests():
    my_grid = [[1, 2, 3], [4, 5, 6]]
    tall = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 0]]
    test_islands = [[1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]

    obj1 = Object(Bounds(0, 0, 4, 4))
    obj2 = Object(Bounds(1, 1, 5, 5))
    obj3 = Object(Bounds(0, 0, 2, 2))

    expect(rotate180([[1, 1, 0], [0, 0, 1]]), [[1, 0, 0], [0, 1, 1]])
    expect(findUnique([obj1, obj2, obj3], lambda obj: area(obj)), obj3)
    global GRID
    GRID = test_islands
    tio = Object(Bounds(0, 0, 4, 4))
    expect(grabPart(tio, 0, 0), test_islands)
    expect(grabPart(tio, 1, 1), [[1, 0], [0, 0]])
    expect(grabPart(tio, 2, 2), [[1, 0], [0, 1]])
    expect(grabPart(tio, 2, 0), [[0, 0, 1, 0], [0, 0, 0, 1]])

    if getPrimaryColor([[3, 1, 0, 0, 0, 3, 0, 2, 3]]) != 3:
        print("Error during startup tests: getPrimaryColor")
    if rotate90([[1, 2], [3, 4]]) != [[3, 1], [4, 2]]:
        print("Error during startup tests: rotate90")
    if newGrid(2, 2) != [[0, 0], [0, 0]]:
        print("Error during startup tests: newGrid")
    if flipVertical([[1, 2], [3, 4]]) != [[3, 4], [1, 2]]:
        print("Error during startup tests: flipVertical")
    if flipHorizontal([[1, 2, 3], [4, 5, 6]]) != [[3, 2, 1], [6, 5, 4]]:
        print("Error during startup tests: flipHorizontal")
    if flipVerticalOverPoint(tall, Point(1, 1)) != [[5, 6], [3, 4], [1, 2], [7, 8], [9, 0]]:
        print("Error during startup tests: flipVerticalOverPoint")
    if findIslands(test_islands, True)[0].X != 1 or findIslands(test_islands, False)[2].Y != 4:
        print("Error during startup tests: findIslands")
    if doubleCanvasSize([[1]]) != [[1, 0], [0, 0]]:
        print("Error during startup tests: doubleCanvasSize")
        print(doubleCanvasSize([[1]]))


runFunctionalTests()

# Type and Operation definitions
#
#  This section defines the primitive types and operations that are used by the program search to try to solve tasks.

In [0]:
class Type:
    def __init__(self, name, values=None, superType=None, dynamic=False):
        '''
        superType is a fallback type if no values are added or if no solution can be
        found using the existing values. Ex: OUTPUT_ONLY_COLOR is a subset of OUTPUT_COLOR
        is a subset of color

        dynamic means that the type cannot be hard coded, it will always be a knob
        and that knob should use the values from the task environment
        '''
        self.name = name
        self.values = values if values is not None else []
        self.superType = superType
        self.dynamic = dynamic


class Operation:
    def __init__(self, name, params, returns):
        self.name = name
        self.params = params
        self.returns = returns


class Lambda:
    def __init__(self, name, returns, provides):
        self.name = name
        self.returns = returns
        self.provides = provides


class Precomp:
    def __init__(self, name, dataType, expression):
        self.name = name
        self.dataType = dataType
        self.expression = expression


# PRECOMPS = [
#    Precomp("islands", "bounds[]"),
#    Precomp(""),
#    Precomp(),

TYPES = [
    Type("grid", []),
    Type("grid[]"),
    Type("bool", [True, False]),
    Type("bounds", ["FULL_BOUNDS", "LARGEST_MONO_OBJECT", "SMALLEST_MONO_OBJECT",
                    "PRIMARY_BOUNDING_BOX", "SECONDARY_BOUNDING_BOX"]),
    Type("bounds[]", ["LAKES", "ISLANDS"]),
    Type("number"),
    Type("nudge", [-1, 0, 1]),
    Type("smallPositiveNumber", [1, 2, 3]),
    Type("smallNumber", [0, 1, 2, 3]),
    Type("2-4", [2, 3, 4]),
    Type("0-2", [0, 1, 2]),
    Type("0-1", [0, 1]),
    Type("dot"),
    Type("dot[]", ["CDOTS", "DOTS"]),
    Type("object", ["LEARNED_OUTPUT_OBJECT", "LARGEST_MONO_OBJECT", "SMALLEST_MONO_OBJECT",
                    "ALL_COLOR_BOUNDS_OBJECT", "BIGGEST_BLOCK_OBJECT", "LINE"]),
    Type("object[]", ["LAKES", "ISLANDS", "MONO_OBJECTS"]),
    Type("point"),
    Type("point[]"),
    Type("sequence", ["SEQUENCE"]),

    Type("color", [0, "BLUE", "RED", "GREEN", "YELLOW", "GRAY", "PINK", "ORANGE", "LIGHTBLUE", "BROWN"]),
    Type("backgroundColor", [0], superType="color"),
    Type("INPUT_COLOR", [], dynamic=True, superType="color"),
    Type("OUTPUT_COLOR", [], dynamic=True, superType="color"),
    Type("OUTPUT_HEIGHT", [], dynamic=True),
    Type("OUTPUT_WIDTH", [], dynamic=True),

    Type("lambda(object):number", ["identity"]), # Identity is technically a function
]
print("Initialized " + str(len(TYPES)) + " types.")

LAMBDAS = [
    Lambda("lambda(bounds):grid", "grid", ["bounds"]),
    Lambda("lambda(dot):grid", "grid", ["dot"]),
    Lambda("lambda(object):grid", "grid", ["object"]),
    Lambda("lambda(object):number", "number", ["object"]),
    Lambda("lambda(INPUT_COLOR):grid", "grid", ["INPUT_COLOR"]),
    Lambda("lambda(bounds,color):grid", "grid", ["bounds", "color"]),
]
print("Initialized " + str(len(LAMBDAS)) + " lambdas.")

'''
 You can stash Operations here if you want to focus the algorithm more
  Operation("flipVerticalOverPoint", ["grid", "point"], "grid"),
  Operation("flipHorizontalOverPoint", ["grid", "point"], "grid"),
  Operation("getLargestSquareBounds", ["grid"], "bounds"),
  Operation("getAllColorBounds", ["grid"], "bounds[]"),
  Operation("doubleCanvasSize", ["grid"], "grid"), # No longer needed in most cases - built into functions that need it
  Operation("kaleidoscopeAroundPoint", ["grid", "point"], "grid"),
'''
OPERATIONS = [
    # Temporarily disabled methods:
    # Operation("newGrid", ["smallPositiveNumber", "smallPositiveNumber"], "grid", ["size"]),

    # # Size-related methods
    Operation("scaleUp", ["2-4"], "grid"),
    Operation("splitVertical", [], "grid[]"),
    Operation("splitHorizontal", [], "grid[]"),
    Operation("slice", ["OUTPUT_HEIGHT", "OUTPUT_WIDTH", "0-1"], "object[]"),

    # Color-mutating methods
    Operation("fillBoundsWithColor", ["bounds", "OUTPUT_COLOR"], "grid"),
    Operation("replaceColorInBounds", ["INPUT_COLOR", "OUTPUT_COLOR", "bounds"], "grid"),
    Operation("replaceColor", ["INPUT_COLOR", "OUTPUT_COLOR"], "grid"),
    Operation("drawDotBorder", ["dot", "OUTPUT_COLOR"], "grid"),
    Operation("drawBorder", ["bounds", "OUTPUT_COLOR"], "grid"),
    Operation("getBoundingBoxOfColor", ["INPUT_COLOR"], "bounds"),

    Operation("shrinkBounds", ["bounds"], "bounds"),
    Operation("growBounds", ["bounds"], "bounds"),
    Operation("rotate90", [], "grid"),
    Operation("rotate180", [], "grid"),
    Operation("joinFourTurns", [], "grid"),
    Operation("clear", [], "grid"),
    Operation("putObject", ["object"], "grid"),
    Operation("fractal", [], "grid"),

    # TODO: Too many different operations all flip things
    Operation("flipVertical", [], "grid"),
    Operation("flipHorizontal", [], "grid"),
    Operation("mirrorVertical", ["0-1"], "grid"),
    Operation("kaleidoscope", [], "grid"),
    Operation("joinMultiUsingAnd", ["grid[]"], "grid"),
    Operation("joinMultiUsingOr", ["grid[]"], "grid"),
    Operation("joinMultiUsingXor", ["grid[]"], "grid"),
    Operation("invert", ["grid", "OUTPUT_COLOR"], "grid"),

    Operation("findSmallest", ["bounds[]"], "bounds"),
    Operation("findLargest", ["bounds[]"], "bounds"),

    Operation("filterDotsByColor", ["dot[]", "INPUT_COLOR"], "dot[]"),
    Operation("filterOutDots", ["bounds[]"], "bounds[]"),
    Operation("sortLargestFirst", ["bounds[]"], "bounds[]"),
    Operation("sortSmallestFirst", ["bounds[]"], "bounds[]"),

    Operation("forEachBounds", ["bounds[]", "lambda(bounds):grid"], "grid"),
    Operation("forEachDot", ["dot[]", "lambda(dot):grid"], "grid"),
    Operation("forEachObject", ["object[]", "lambda(object):grid"], "grid"),
    Operation("forEachInputColor", ["lambda(INPUT_COLOR):grid"], "grid"),

    # TODO: BIG ISSUE: This becomes order dependent but I cannot generate order
    Operation("doBoth", ["grid", "grid"], "grid"),

    Operation("drawLine", ["dot", "nudge", "nudge"], "grid"),
    Operation("drawColoredLine", ["dot", "nudge", "nudge", "OUTPUT_COLOR"], "grid"),
    Operation("drawLineFromObject", ["object", "nudge", "nudge"], "grid"),
    Operation("drawLineAwayFromObject", ["dot", "object"], "grid"),
    Operation("drawColoredLineFromObject", ["object", "nudge", "nudge", "OUTPUT_COLOR"], "grid"),

    Operation("moveDotToObject", ["dot", "object"], "grid"),

    # TODO: URGENT to handle the color specificity/subclass problem ASAP
    # Operation("connectColinearPoints", ["dot[]", "OUTPUT_COLOR"], "grid"),
    Operation("connectColinearPoints", ["dot[]", "INPUT_COLOR"], "grid"),
    Operation("grabPart", ["object", "0-2", "0-2"], "grid"),
    Operation("cropToObject", ["object"], "grid"),

    Operation("translateObject", ["object", "smallNumber", "smallNumber"], "grid"),
    Operation("moveObjectToTouchObject", ["object", "object"], "grid"),
    Operation("getCenterPointOf", ["bounds", "bool"], "point"),
    Operation("drawObjectCenteredOnPoint", ["object", "point"], "grid"),

    Operation("getColoredObjects", ["INPUT_COLOR"], "object[]"),
    Operation("getColoredObject", ["INPUT_COLOR"], "object"),

    Operation("findUnique", ["object[]", "lambda(object):number"], "object"),
    Operation("countColors", ["object"], "number"),
    Operation("area", ["object"], "number"),

    Operation("cropToBounds", ["bounds"], "grid"),

    Operation("repeatTile", ["object", "smallPositiveNumber", "smallPositiveNumber"], "grid"),

    # TODO: I keep postponing work to actually fix my type system
    # But at least for color, the purpose of the type system was for performance gains...
    # There are two issues: colors and
    Operation("applyColorSequence", ["bounds[]", "color[]", "lambda(bounds,color):grid"], "grid"),

    # TODO: Support rendering an outputbased on OUTPUT_WIDTH for VERTICAL sequences
    Operation("renderSequence", ["sequence"], "grid"),
]



EXCLUSION_DICT = {
    "size": ["scaleUp", "splitVertical", "splitHorizontal", "kaleidoscope",
             "joinFourTurns", "cropToBounds", "cropToObject", "repeatTile",
             "mirrorVertical"],
    "too_many_dots": ["forEachDot", "connectColinearPoints"],
    "rotation": ["joinFourTurns"] if ENABLE_SYMMETRY_EXCLUSIONS else [],
    "bilateral": ["kaleidoscope"] if ENABLE_SYMMETRY_EXCLUSIONS else [],
    "io_rotation": ["rotate90", "rotate180"],
    "io_bilateral": ["flipHorizontal", "flipVertical"],
}

print("Initialized " + str(len(OPERATIONS)) + " operations.")

programming" approach to solving tasks. The goal of this notebook will be to eventually implement a useful and
powerful genetic algorithm, but all I've implemented so far is a semi-exhaustive tree search across program
structures without any sort of scoring except "yes"/"no" answers to "does this solve the training task?"
#
he approach is inspired somewhat by the MOSES system (at least, the concept of knobs). More generally however,
this code is roughly translated from a more generic Genetic Programming system I developed while researching GP
many years ago. That system was originally written in JavaScript and I couldn't find a robust way to integrate
those libraries into a Kernel that would commit successfully in a Kernels-Only "Code" competition.
#
Since Python has clearly won in the Data Science community ("R" is great too!), I figured it would be worth doing
this translation, since the code will be easier to update, share with the community and integrate into the larger
space of ML. I have some ambitious ideas about how genetic programming concepts can be combined with deep learning
approaches for very flexible learning systems.

In [0]:
class ProgramNode:
    def __init__(self, nodeType, name, args=None, dataType=None):
        self.nodeType = nodeType
        self.name = name
        self.args = args if args is not None else []
        self.dataType = dataType
        self.complexity = None
        self.complexity = computeProgramComplexity(self)

        self.locals = []  # The locals provided by a lambda
        self.knobName = ""

        self.hasKnobInfo = False  # Set to true when the tree is visited
        self.hasDynamicValues = False

    def __str__(self):
        return makeTemplateStringFromProgramNode(self)

    def __repr__(self):
        return "ProgramNode[" + self.__str__() + "]"


class Local:
    def __init__(self, dataType, name):
        self.dataType = dataType
        self.name = name


def simpleProgramStr(p):
    myStr = p.name
    if p.nodeType == "operation" or p.nodeType == "lambda":
        argL = []
        for a in p.args:
            argL.append(simpleProgramStr(a))

        myStr = myStr + "(" + ", ".join(argL) + ")"
    return myStr


def findOperationNames(programNode):
    names = []
    if programNode.nodeType == "operation":
        names.append(programNode.name)
        for arg in programNode.args:
            vals = findOperationNames(arg)
            for v in vals:
                names.append(v)
    return names


class Template:
    def __init__(self, programNode, knobDict, templateString):
        self.programNode = programNode
        self.knobDict = knobDict
        self.templateString = templateString
        self.memoId = ""
        self.operationNames = findOperationNames(self.programNode)

    def __str__(self):
        return "Template: " + str(self.programNode) + "\nknobDict" + str(
            self.knobDict) + "\n" + "templateString: " + self.templateString


class Constraints:
    def __init__(self, returns, consumes=None):
        self.returns = returns
        self.consumes = consumes


class TrainingPair:
    def __init__(self, input, output):
        self.input = input
        self.output = output

# Static Program Generation Entrypoint

In [0]:
# Global variable that holds the current training grid. There are probably better ways to provide this information
# to operations which need it, but I'm racing to get on the leaderboard while 4/100 is still a good score.

ALL_PROGRAMS = []  # Static
ALL_TEMPLATES = []  # Static


def buildPrograms():
    global ALL_PROGRAMS
    global ALL_TEMPLATES
    ALL_PROGRAMS = generateAllProgramTreesUpToDepthN(MAX_SEARCH_DEPTH, MAX_COMPLEXITY, Constraints(returns="grid"), [])
    ALL_PROGRAMS = list(map(copy.deepcopy, ALL_PROGRAMS))
    ALL_PROGRAMS = sortPrograms(ALL_PROGRAMS)
    ALL_TEMPLATES = buildTemplatesAndMemo(ALL_PROGRAMS)

    if DEBUG_LEVEL >= 1:
        for p in ALL_TEMPLATES:
            print(p.templateString)


def buildTemplatesAndMemo(programs):
    templates = []
    for program in programs:
        template = makeTemplateFromProgram(program)
        if template is None or len(template.knobDict) > MAX_KNOBS:
            if DEBUG_LEVEL >= 1:
                print("Skipping program " + str(program))
            continue

        addToMemoIfAble(template)

        templates.append(template)
    return templates


def sortPrograms(programs):
    scored_programs = list(map(lambda p: {"program": p, "complexity": computeProgramComplexity(p)}, programs))
    scored_programs = sorted(scored_programs, key=lambda x: x["complexity"])
    return list(map(lambda p: p["program"], scored_programs))


MEMO_ID_COUNTER = 0
MEMO_IDS = {}  # Static: map from an input function template string to a variable name that can read it from the program
MEMO_VALUES = {}  # A map from memoId to variable values. Reset for each training input


def getNextMemoId():
    global MEMO_ID_COUNTER
    MEMO_ID_COUNTER += 1
    return "memo" + str(MEMO_ID_COUNTER)


def addToMemoIfAble(template):
    if not ENABLE_MEMO:
        return

    if template.templateString in MEMO_IDS:
        template.memoId = MEMO_IDS[template.templateString]
        return

    # To be a memo it must have no knobs, no lambdas and at least one function call
    if len(template.knobDict) == 0 and "lambda" not in template.templateString and "(" in template.templateString:
        new_memo_id = getNextMemoId()
        MEMO_IDS[template.templateString] = new_memo_id
        template.memoId = new_memo_id


# Removes all the evaluated values from the memo while keeping the keys
def resetMemoValues():
    global MEMO_VALUES
    MEMO_VALUES = {}


LEARNED_OUTPUT_OBJECT = Object(Bounds(0, 0, 0, 0))
ENV = dict(variables={})
INPUT_GRID = [[0]]
GRID = [[0]]
LAKES = []
ISLANDS = []
XISLANDS = []
DOTS = []
CDOTS = []
BG_COLOR = 0
FULL_BOUNDS = Bounds(0, 0, 1, 1)
LINE = None

PRIMARY_COLOR = 0
SECONDARY_COLOR = 0
INPUT_COLORS = []
PRIMARY_BOUNDING_BOX = Bounds(0, 0, 1, 1)
SECONDARY_BOUNDING_BOX = Bounds(0, 0, 1, 1)
COLORED_OBJECTS = [[], [], [], [], [], [], [], [], [], []]
LARGEST_MONO_OBJECT = Object(Bounds(0, 0, 1, 1))
SMALLEST_MONO_OBJECT = Object(Bounds(0, 0, 1, 1))
MONO_OBJECTS = []
ALL_COLOR_BOUNDS_OBJECT = Object(Bounds(0, 0, 1, 1))
BIGGEST_BLOCK_OBJECT = Object(Bounds(0, 0, 1, 1))
SEQUENCE = None

# CONSTANTS
BG = 0
BLUE = 1
RED = 2
GREEN = 3
YELLOW = 4
GRAY = 5
PINK = 6
ORANGE = 7
LIGHTBLUE = 8
BROWN = 9

G = {} # G Is the global variable storage
def setTaskwideValues(task):
    global LEARNED_OUTPUT_OBJECT, ENV
    conclusions = reasonAboutTask(task)

    output_grid = task["train"][0]["output"]
    all_bounds = findIslands(output_grid)
    # print("Found " + str(len(all_bounds)) + " bounds")
    largest_bound = findLargest(all_bounds)
    # print("Found a largest bound " + str(largest_bound))
    objectsList = objectifyBoundsList([largest_bound], output_grid)

    LEARNED_OUTPUT_OBJECT = objectsList[0] if len(objectsList) > 0 else Object(Bounds(0, 0, 0, 0))

    for var in conclusions["variables"]:
        ENV["variables"][var.name] = var.values
        if DEBUG_LEVEL >= 2:
            print("Using " + var.name + " values: " + str(var.values))

    conclusions["precomps"] = []
    for pair in task["train"]:
        precomp = getPrecomputedValues(pair["input"])
        conclusions["precomps"].append(precomp)

    return conclusions


def transpose(mat):
    return [list(row) for row in zip(mat)]


def readGridAsSequenceElements(grid, direction):
    if direction == "VERTICAL":
        return clone2d(grid)
    else:
        return transpose(grid)


def elementsHavePeriod(elements, period):
    if period >= len(elements):
        return False

    for i in range(period, len(elements)):
        element = elements[i]
        if element != elements[i % period]:
            return False
    return True


def findMinimumPeriod(elements):
    for period in range(1, len(elements) - 1):
        if elementsHavePeriod(elements, period):
            return period
    return len(elements)


def findSequence(grid):
    direction = ""
    if len(grid) > len(grid[0]):
        direction = "VERTICAL"
    elif len(grid[0]) > len(grid):
        direction = "HORIZONTAL"
    else:
        return None
    
    elements = readGridAsSequenceElements(grid, direction)
    period = findMinimumPeriod(elements)

    sequence = Sequence(elements, period, direction)

    return sequence


def getPrecomputedValues(input_grid):
    global GRID
    GRID = input_grid

    primary_color = getPrimaryColor(input_grid)
    secondary_color = getSecondaryColor(input_grid)
    colored_objects = findColoredObjects(input_grid)
    blocks = findBiggestBoxes(input_grid)
    islands = findIslands(input_grid)
    lines = filterToLines(islands)
    sequence = findSequence(input_grid)

    g = {
        "INPUT_GRID": input_grid,
        "ISLANDS": islands,
        "XISLANDS": findIslands(input_grid, True),
        "LAKES": findLakes(input_grid, False),
        "DOTS": findDots(input_grid),
        "CDOTS": findCDots(input_grid),
        "PRIMARY_COLOR": primary_color,
        "SECONDARY_COLOR": secondary_color,
        "COLORED_OBJECTS": colored_objects,
        "PRIMARY_BOUNDING_BOX": getBoundingBoxOfColor(primary_color),
        "SECONDARY_BOUNDING_BOX": getBoundingBoxOfColor(secondary_color),
        "ALL_COLOR_BOUNDS_OBJECT": Object(getBoundingBoxOfEverything(input_grid), input_grid),
        "FULL_BOUNDS": Bounds(0, 0, len(input_grid), len(input_grid[0])),
        "BIGGEST_BLOCK_OBJECT": findLargest(blocks),
        "LINE": lines[0] if len(lines) > 0 else None,
        "SEQUENCE": sequence
    }

    premo = getPrecomputedMonoObjects(colored_objects)
    g["MONO_OBJECTS"] = premo["MONO_OBJECTS"]
    g["LARGEST_MONO_OBJECT"] = premo["LARGEST_MONO_OBJECT"]
    g["SMALLEST_MONO_OBJECT"] = premo["SMALLEST_MONO_OBJECT"]

    return g


# DEPRECATED: Use getPrecomputedValues instead which caches to 'g'
def setPrecomputedValues(g):
    global GRID, INPUT_GRID, ISLANDS, XISLANDS, LAKES, DOTS, CDOTS, PRIMARY_COLOR, SECONDARY_COLOR
    global COLORED_OBJECTS, LARGEST_MONO_OBJECT, ALL_COLOR_BOUNDS_OBJECT, PRIMARY_BOUNDING_BOX, SECONDARY_BOUNDING_BOX
    global LARGEST_MONO_OBJECT, SMALLEST_MONO_OBJECT, FULL_BOUNDS, BIGGEST_BLOCK_OBJECT
    global LINE, MONO_OBJECTS, SEQUENCE
    INPUT_GRID = g["INPUT_GRID"]  # Read as a global by some param-invariant operations
    ISLANDS = g["ISLANDS"]
    XISLANDS = g["XISLANDS"]
    LAKES = g["LAKES"]
    DOTS = g["DOTS"]
    CDOTS = g["CDOTS"]
    PRIMARY_COLOR = g["PRIMARY_COLOR"]
    SECONDARY_COLOR = g["SECONDARY_COLOR"]
    COLORED_OBJECTS = g["COLORED_OBJECTS"]
    PRIMARY_BOUNDING_BOX = g["PRIMARY_BOUNDING_BOX"]
    SECONDARY_BOUNDING_BOX = g["SECONDARY_BOUNDING_BOX"]
    ALL_COLOR_BOUNDS_OBJECT = g["ALL_COLOR_BOUNDS_OBJECT"]
    MONO_OBJECTS = g["MONO_OBJECTS"]
    LARGEST_MONO_OBJECT = g["LARGEST_MONO_OBJECT"]
    SMALLEST_MONO_OBJECT = g["SMALLEST_MONO_OBJECT"]
    FULL_BOUNDS = g["FULL_BOUNDS"]
    BIGGEST_BLOCK_OBJECT = g["BIGGEST_BLOCK_OBJECT"]
    LINE = g["LINE"]
    SEQUENCE = g["SEQUENCE"]

# Program Execution & Evaluation

In [0]:


def isTemplateExcluded(template, exclusions):
    for exName in exclusions:
        for item in EXCLUSION_DICT[exName]:
            if item in template.operationNames:
                return True
    return False


def innerSearch(task, debug=False):
    programs_generated_count = 0
    instances_evaluated = 0

    trainIn = task["train"][0]["input"]
    trainOut = task["train"][0]["output"]

    # The lifetime of the memo is a single problem (against train[0] only)
    resetMemoValues()
    conclusions = setTaskwideValues(task)
    setPrecomputedValues(conclusions["precomps"][0])

    exclusion_count = 0
    for template in ALL_TEMPLATES:
        if DEBUG_LEVEL >= 3:
            print(template.templateString)

        if ENABLE_EXCLUSIONS and isTemplateExcluded(template, conclusions["exclusions"]):
            exclusion_count += 1
            continue

        valid_knob_values_list = findAllValidTemplateInstances(template, trainIn, trainOut)
        for valid_knob_values in valid_knob_values_list:
            full_solution = tryToPassAllTraining(template, valid_knob_values, task, conclusions, debug)
            if full_solution is not None:
                if DEBUG_LEVEL >= 1:
                    print("Solution passed all examples. Using as final.")
                print(full_solution)
                return full_solution
            else:
                setPrecomputedValues(conclusions["precomps"][0])
                if DEBUG_LEVEL >= 2:
                    print("Failed on further examination :(")

    print("Excluded " + str(exclusion_count) + " nodes")


def findAllValidTemplateInstances(template, trainIn, trainOut):
    if len(template.knobDict) > 0:
        validSelections = []
        all_options = cartesianProductDict(template.knobDict)
        for selection in all_options:
            solution = tryInstance(template, selection, trainIn, trainOut)
            if solution is not None:
                validSelections.append(selection)
        return validSelections
    else:
        solution = tryInstance(template, {}, trainIn, trainOut)
        if solution is not None:
            return [{}]
        return []


def makeSolutionInstance(templateString, knobSelections):
    solution_string = templateString
    if DEBUG_LEVEL >= 2:
        print("About to try template " + solution_string + " with knobs " + str(knobSelections.items()))
    for key, value in knobSelections.items():
        solution_string = solution_string.replace(key, str(value))

    return solution_string


# Try the template and return a valid evaluatable solution string if it passes in/out
def tryInstance(template, knobSelections, trainIn, trainOut):
    solution_instance = makeSolutionInstance(template.templateString, knobSelections)
    if DEBUG_LEVEL >= 2:
        print(solution_instance)

    result = executeSolution(solution_instance, trainIn)

    if template.memoId is not None and len(knobSelections) == 0:
        MEMO_VALUES[template.memoId] = result

    if result == trainOut:
        return solution_instance
    return None


# solution is assumed to be an executable string
def executeSolution(solution, input_grid, islands=None, lakes=None):
    global GRID
    GRID = clone2d(input_grid)
    return eval(solution)


# TODO: Update to score and choose the 3 best functions if there isn't a 100% success
# Note: uses a template and not a solution because it needs to regenerate the program
def tryToPassAllTraining(template, knobValues, task, conclusions, debug=False):
    newTemplateString = makeTemplateStringFromProgramNode(template.programNode, True)
    full_solution_instance = makeSolutionInstance(newTemplateString, knobValues)

    for i in range(len(task["train"])):
        pair = task["train"][i]
        setPrecomputedValues(conclusions["precomps"][i])
        result = executeSolution(full_solution_instance, pair["input"])
        if result != pair["output"]:
            if DEBUG_LEVEL >= 3:
                print("Solution " + str(full_solution_instance) + " failed on input #" + str(i))
            return None
    return full_solution_instance

# Program Generation

In [0]:
def getSingleNodeComplexity(node):
    complexity = 1
    if node.nodeType == "operation":
        complexity = 2
    if node.nodeType == "lambda":
        complexity = 0
    if node.nodeType == "literal":
        complexity: 0.5
    return complexity


def computeProgramComplexity(node):
    if node.complexity is not None:
        return node.complexity
    complexity = getSingleNodeComplexity(node)
    if node.args is not None:
        for arg in node.args:
            complexity += computeProgramComplexity(arg)
    return complexity


def cartesianProduct(inputList):
    return itertools.product(*inputList)


def cartesianProductDict(knobDict):
    listOfKeys = []
    listOfLists = []
    for key, value in knobDict.items():
        valueList = value # This might not be a list yet!
        if isinstance(value, str):
            valueList = ENV["variables"][value]

        listOfKeys.append(key)
        listOfLists.append(valueList)

    product = itertools.product(*listOfLists);

    def pToDict(valueList):
        result = {}
        for i in range(len(valueList)):
            result[listOfKeys[i]] = valueList[i]
        return result

    return map(pToDict, product)


def listFind(input_list, callback):
    results = list(filter(callback, input_list))
    if len(results) > 0:
        return results[0]
    return None


def lookupParamsForOperation(operation):
    itemHolder = [op for op in OPERATIONS if op.name == operation.name]
    return itemHolder[0].params


def lookupLambdaFromNode(inputLambda):
    itemHolder = [fn for fn in LAMBDAS if fn.name == inputLambda.name]
    return itemHolder[0]


# TODO: Prevent nesting of these functions:'scaleUp', 'doBoth' (add doThree')?
# TODO: Prevent doBoth from taking a literal (ex: GRID)

def generateAllProgramTreesUpToDepthN(depth, complexity_budget, constraints, available_locals=[]):
    pad = ("    " * (MAX_SEARCH_DEPTH - depth))
    if depth <= 0 or complexity_budget <= 0:
        return []

    programs = []
    availableNodes = findNodesMatchingConstraints(constraints, available_locals)
    for node in availableNodes:
        if DEBUG_LEVEL >= 3: print(pad + "Trying node " + str(node))
        if node.nodeType == "variable" or node.nodeType == "literal":
            if complexity_budget > getSingleNodeComplexity(node):
                programs.append(node)
        # TODO: Update this when there are zero-arg functions
        elif node.nodeType == "operation":
            param_options = []
            expected_params = lookupParamsForOperation(node)
            depth_cost = 1  # + (len(expected_params) - 1) * 0.1
            complexity_left = complexity_budget - getSingleNodeComplexity(node)
            for param in expected_params:
                if DEBUG_LEVEL >= 3: print(pad + " Finding options for Param " + str(param))
                # TODO(P2): Allow consumption of params at a lower level
                subtrees = generateAllProgramTreesUpToDepthN(depth - depth_cost, complexity_left,
                                                             Constraints(returns=param),
                                                             available_locals)
                if DEBUG_LEVEL >= 3: print(pad + " Options:  " + str(subtrees))
                param_options.append(subtrees)
            if len(param_options) > 0:
                allParamOptionCombinations = cartesianProduct(param_options)
                for combo in allParamOptionCombinations:
                    program = ProgramNode("operation", node.name, args=combo)
                    if program.complexity < complexity_budget:
                        if DEBUG_LEVEL >= 3: print(pad + "Using param option " + str(program))
                        programs.append(program)
                    else:
                        if DEBUG_LEVEL >= 3: print(pad + "Skipping param option due to complexity")
            elif len(expected_params) == 0:
                programs.append(ProgramNode("operation", node.name, []))
            else:

                if DEBUG_LEVEL >= 3: print(pad + "No param options found for operation node " + node.name)
        elif node.nodeType == "lambda":
            fnMetadata = lookupLambdaFromNode(node)
            # TODO: Allow multiple lambda nestings, eventually
            available_locals = getLocalsFromProvides(fnMetadata.provides)
            node.locals = available_locals
            complexity_left = complexity_budget - getSingleNodeComplexity(node)
            depth = depth if SKIP_LAMBDA_DEPTH == False else depth + 1
            subtrees = generateAllProgramTreesUpToDepthN(depth, complexity_left, Constraints(returns=fnMetadata.returns,
                                                                                             consumes=fnMetadata.provides),
                                                         available_locals)
            for subtree in subtrees:
                program = ProgramNode("lambda", node.name, args=[subtree])
                if not doesSubtreeConsumeAllArguments(program, fnMetadata.provides):
                    if DEBUG_LEVEL >= 3: print("Skipping subtree that fails to consume args")
                    continue;
                if program.complexity < complexity_budget:
                    program.locals = node.locals
                    programs.append(program)

    fitting_programs = list(filter(lambda p: p.complexity < complexity_budget, programs))
    if len(fitting_programs) != len(programs):
        print("Filtered " + str(len(programs)) + " down to " + str(len(fitting_programs)))
    return fitting_programs


def doesSubtreeConsumeAllArguments(programNode, provides):
    allUsed = listAllValuesConsumedByProgram(programNode)
    # print("Comparing " + str(provides) + " " + str(allUsed))
    for p in provides:
        if p not in allUsed:
            return False
    return True


def listAllValuesConsumedByProgram(programNode):
    if programNode.nodeType == "literal":
        return [programNode.dataType]
    result = [];
    for arg in programNode.args:
        result = result + listAllValuesConsumedByProgram(arg)

    return result


AVAILABLE_LOCALS = []
GLOBAL_LOCAL_COUNTER = 0


def getLocalsFromProvides(typeList):
    available_locals = []
    for t in typeList:
        global GLOBAL_LOCAL_COUNTER
        available_locals.append(Local(t, t + str(GLOBAL_LOCAL_COUNTER)))
        GLOBAL_LOCAL_COUNTER += 1

    return available_locals


def findNodesMatchingConstraints(constraints, available_locals=[]):
    nodes = []
    for x in available_locals:
        if x.dataType == constraints.returns:
            nodes.append(ProgramNode("literal", x.name, dataType=x.dataType))

    # TODO(P1): Currently we don't use variables if literals are available for now
    for t in TYPES:
        if typeMatchesConstraints(t, constraints):
            if len(t.values) <= 2 and not t.dynamic:
                for v in t.values:
                    literal = ProgramNode("literal", v, dataType=t.name)
                    nodes.append(literal)
            else:
                node = ProgramNode("variable", t.name, dataType=t.name)
                nodes.append(node)
                if t.dynamic:
                    node.hasDynamicValues = True

    for op in OPERATIONS:
        if operationMatchesConstraints(op, constraints):
            nodes.append(ProgramNode("operation", op.name, []))

    # Don't allow nested lambdas
    if len(available_locals) == 0:
        for fn in LAMBDAS:
            if lambdaMatchesConstraints(fn, constraints):
                nodes.append(ProgramNode("lambda", fn.name, []))
    return nodes


def typeMatchesConstraints(t, constraints):
    return t.name == constraints.returns and (len(t.values) > 0 or t.dynamic);


def operationMatchesConstraints(op, constraints):
    # if (constraints.consumes != None):
    # for c in constraints.consumes:
    #    if (c not in op.params):
    #            return False

    return op.returns == constraints.returns


def lambdaMatchesConstraints(fn, constraints):
    return fn.name == constraints.returns

# Template generation

In [0]:
def makeTemplateFromProgram(programNode):
    knobDict = {}
    if addKnobInfoToTree(programNode, knobDict):
        templateString = makeTemplateStringFromProgramNode(programNode)
        template = Template(programNode, knobDict, templateString)
        return template
    else:
        print("failed to add knobInfo to program node " + str(programNode))


def makeTemplateStringFromProgramNode(node, disableMemo=False):
    if node is None:
        return ""
    if node.nodeType == "literal":
        return str(node.name)
    if node.nodeType == "variable":
        return node.knobName or node.name
    if node.nodeType == "operation":
        args = map(lambda arg: makeTemplateStringFromProgramNode(arg), node.args)

        arg_strs = map(lambda arg: returnMemoValueExpressionIfAvailable(arg, disableMemo), args)
        return node.name + "(" + ", ".join(arg_strs) + ")"
    if node.nodeType == "lambda":
        lookupLambdaFromNode(node)
        provided_local_names = list(map(lambda loc: loc.name, node.locals))
        args = makeTemplateStringFromProgramNode(node.args[0]) if len(node.args) > 0 else ""
        return "lambda " + ','.join(provided_local_names) + ": " + args


def returnMemoValueExpressionIfAvailable(templateString, disableMemo):
    if not disableMemo and templateString in MEMO_IDS:
        return "MEMO_VALUES[\"" + MEMO_IDS[templateString] + "\"]"
    return templateString


# Return true iff the tree could be fully populated with valid knob values
# this ensures that we don't build any constraint-violating trees
def addKnobInfoToTree(node, knobDict):
    if not node:
        return False
    if node.hasKnobInfo is True:
        node.printThis().shouldntHappen
    node.hasKnobInfo = True
    if node.nodeType == "literal":
        return True
    if node.nodeType == "variable":
        values = findValuesForType(node.name)
        if node.hasDynamicValues or len(values) > 0:
            node.knobName = "knob" + str(len(knobDict)) + "~"
            knobDict[node.knobName] = values
            return True
        else:
            return False
    elif node.nodeType == "operation" or node.nodeType == "lambda":
        success = True
        for arg in node.args:
            success = success and addKnobInfoToTree(arg, knobDict)
        return success
    # print("node somehow matched a non-node?")
    return False


def findValuesForType(paramType):
    index = TYPES

    typeItem = listFind(TYPES, lambda t: t.name == paramType)
    if typeItem is not None:
        if typeItem.dynamic:
            return typeItem.name

        return typeItem.values

    return []

## Unit Tests for the system
# # These tests are used during development and maintenance to make sure core functionality continues to work

In [0]:
def runSystemTests():
    expect(len(findLakes([[1, 0, 1], [1, 0, 1], [1, 1, 1], [1, 1, 1], [1, 0, 1], [1, 1, 1]], False)), 1)
    expect(joinMultiUsingAnd([[[0, 1], [1, 1]], [[0, 1], [1, 0]]]), [[0, 1], [1, 0]])
    expect(str(findSmallest([Bounds(3, 3, 6, 5), Bounds(4, 4, 5, 5), Bounds(0, 0, 1, 2)])), "Bounds(4, 4, 5, 5)")
    expect(str(findLargest([Bounds(3, 3, 6, 5), Bounds(4, 4, 5, 5), Bounds(0, 0, 1, 2)])), "Bounds(3, 3, 6, 5)")

    single_variable = ProgramNode("variable", "bool")
    single_arg = ProgramNode("operation", "single_arg", [single_variable])
    more_complex = ProgramNode("operation", "more_complex", [single_arg, single_variable])

    inner_lambda = ProgramNode("lambda", "lambda(grid,bounds):grid", [single_variable])
    inner_lambda.locals = [Local("grid", "grid0"), Local("bounds", "bounds1")]
    with_lambda = ProgramNode("operation", "with_lambda", [inner_lambda])

    expect(computeProgramComplexity(single_variable), 1)
    expect(computeProgramComplexity(single_arg), 3)
    expect(computeProgramComplexity(more_complex), 6)
    expect(sortPrograms([single_arg, more_complex, single_variable]), [single_variable, single_arg, more_complex])
    # Note: in practice 'bool' should never appear, instead a literal or knob should be used
    expect(makeTemplateStringFromProgramNode(single_arg), "single_arg(bool)")
    expect(makeTemplateStringFromProgramNode(more_complex), "more_complex(single_arg(bool), bool)")
    expect(executeSolution("rotate90(GRID)", [[1, 2], [3, 4]]), [[3, 1], [4, 2]])

    # test_knob_dict = {}
    # expect(addKnobInfoToTree(more_complex, test_knob_dict), True)
    # expect(test_knob_dict, [[True, False], [True, False]])

    # template = makeTemplateFromProgram(single_arg)
    # expect(template.knobDict, { "knob0~": [True, False]})
    # expect(template.templateString, "single_arg(knob0~)")
    # expect(makeTemplateStringFromProgramNode(with_lambda), "with_lambda(lambda grid0,bounds1: knob0~)")


runSystemTests()

without knowing the specifics of how the competition is structured

In [0]:
def computePredictionForTask(task, outputId, useDebugSolution="", debug=False):
    global EXECUTION_COUNT
    EXECUTION_COUNT += 1
    if EXECUTION_COUNT > EXECUTION_COUNT_LIMIT:
        return [None, None, outputId]

    training_pairs_raw = task['train']
    test_grid = task['test'][0]['input']

    training_pairs = []
    for p in training_pairs_raw:
        pair = TrainingPair(p["input"], p["output"])
        training_pairs.append(pair)

    if ENABLE_SKIP_LARGE_ITEMS and len(test_grid) > 24 and len(test_grid[0]) > 24:
        print("Skipping task which is too big for performance reasons!")
        return [None, None, outputId]

    solution = useDebugSolution or innerSearch(task, debug)

    if solution:
        if debug:
            print("found a solution. Trying it: " + solution)
        conclusions = setTaskwideValues(task)
        setPrecomputedValues(getPrecomputedValues(test_grid))

        result = executeSolution(solution, test_grid)
        return [result, solution, outputId]
    else:
        if debug:
            print("Could not find a solution")
        return [None, None, outputId]


def trySolveTaskByFile(path, filename, useSolution="", debug=False):
    task_file = str(path / filename)
    with open(task_file, 'r') as f:
        task = json.load(f)
        return trySolveTask(task, useSolution=useSolution, debug=debug)


def getInputGridFor(path, filename):
    task_file = str(path / filename)
    with open(task_file, 'r') as f:
        task = json.load(f)
        return task['train'][0]['input']


def trySolveTask(task, useSolution="", debug=False):
    [prediction, solution, outputId] = computePredictionForTask(task, "", useDebugSolution=useSolution, debug=debug)
    doesMatch = prediction == task['test'][0]['output']

    if DEBUG_LEVEL >= 1:
        if not doesMatch:
            print("Pred: " + str(prediction))
            print("Actl: " + str(task['test'][0]['output']))
        print("Matches? " + str(doesMatch))
    result = solution if doesMatch else None
    return result

# Competition Harness
# # Read/write files and submission output

In [0]:
def flattener(prediction):
    str_pred = str([row for row in prediction])
    str_pred = str_pred.replace(', ', '')
    str_pred = str_pred.replace('[[', '|')
    str_pred = str_pred.replace('][', '|')
    str_pred = str_pred.replace(']]', '|')
    return str_pred


def plot_task(task):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9', '#FF4136', '#2ECC40', '#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    fig, axs = plt.subplots(1, 3, figsize=(15, 15))
    axs[0].imshow(task['train'][0]['input'], cmap=cmap, norm=norm)
    axs[0].axis('off')
    axs[0].set_title('Train Input')
    axs[1].imshow(task['train'][0]['output'], cmap=cmap, norm=norm)
    axs[1].axis('off')
    axs[1].set_title('Train Output')
    axs[2].imshow(task['test'][0]['input'], cmap=cmap, norm=norm)
    axs[2].axis('off')
    axs[2].set_title('Test Input')
    # axs[3].imshow(task['test'][0]['output'], cmap=cmap, norm=norm)
    # axs[3].axis('off')
    # axs[3].set_title('Test Output')
    plt.tight_layout()
    plt.show()

In [0]:

training_path = data_path / 'training'
evaluation_path = data_path / 'evaluation'
test_path = data_path / 'test'

def runPredictionsAndGenerateSubmission():
    submission = pd.read_csv(data_path / 'sample_submission.csv', index_col='output_id')
    attempted = 0
    solved = 0

    def handleResult(predsolid):
        [prediction, solution, outid] = predsolid
        nonlocal attempted
        nonlocal solved
        attempted += 1
        if prediction is not None:
            solved = solved + 1
            print("Task solved. " + str(solved) + "/" + str(attempted))
        else:
            print("Task failed. " + str(solved) + "/" + str(attempted))
            prediction = [[0]]

        prout = flattener(prediction)
        prediction_text = prout + ' ' + prout + ' ' + prout + ' '

        submission.loc[outid, 'output'] = prediction_text

    pool = ProcessPoolExecutor(4)
    futures = []

    main_start = time.time()

    print("Building programs")
    buildPrograms()
    print("Built " + str(len(ALL_PROGRAMS)) + " static programs")

    for output_id in submission.index:
        if attempted > NUM_TAKS_TO_TRY:
            break

        task_id = output_id.split('_')[0]
        pair_id = int(output_id.split('_')[1])
        f = str(test_path / str(task_id + '.json'))
        with open(f, 'r') as read_file:
            task = json.load(read_file)

        #
        # plot_task(task)

        training_pairs_raw = task['train']
        test_grid = task['test'][pair_id]['input']

        if ENABLE_THREADING:
            futures.append(pool.submit(computePredictionForTask, task, output_id))
        else:
            print("Attempting task " + task_id)
            handleResult(computePredictionForTask(task, output_id))

    if ENABLE_THREADING:
        for completed in as_completed(futures):
            predsolid = completed.result()
            handleResult(predsolid)


    print("Attempted " + str(attempted) + " problems:")
    print("Solved " + str(solved) + "/" + str(attempted) + ".")

    main_end = time.time()
    print("Completed in " + str(main_end - main_start) + "s")

    submission.to_csv('submission.csv')


if RUN_TYPE == "MAIN":
    runPredictionsAndGenerateSubmission()

In [0]:

def generateProgramsTimed():
    print("Generating programs")
    build_start = time.time()
    buildPrograms()
    build_end = time.time()
    print("Built " + str(len(ALL_TEMPLATES)) + " templates in " + str(build_end - build_start) + "s")


def tryToSolveTrainingTasks(task_path, tasks, solutions=None):
    generateProgramsTimed()

    pool = ProcessPoolExecutor(4)
    futures = []

    attempts = 0
    solves = 0
    start_time = time.time()
    found_solutions = {}
    for i in range(len(tasks)):
        if attempts > NUM_TAKS_TO_TRY:
            break

        if FOCUS_TASK is not None and tasks[i] != FOCUS_TASK:
            continue

        step_start = time.time()
        task_file = str(task_path / tasks[i])

        with open(task_file, 'r') as f:
            task = json.load(f)

        print("Attempting " + tasks[i])
        if solutions:
            print("Expected solution will be '" + solutions[i] + "'.")
        # plot_task(task)

        if ENABLE_THREADING:
            futures.append(pool.submit(trySolveTask, task))
        else:
            solution = trySolveTask(task, debug=True)
            attempts = attempts + 1
            if solution is not None:
                solves = solves + 1
                print("Solved " + tasks[i] + ". now " + str(solves) + "/" + str(attempts))
                found_solutions[tasks[i]] = solution
            else:
                print("Failed " + tasks[i] + ". now " + str(solves) + "/" + str(attempts))
            print("Completed in " + str(time.time() - step_start) + "s")
            print("")

    if ENABLE_THREADING:
        for x in as_completed(futures):
            attempts += 1
            didSolve = x.result()
            if didSolve:
                solves = solves + 1
                print("Solved! now " + str(solves) + "/" + str(attempts))
            else:
                print("Failed. now " + str(solves) + "/" + str(attempts))

    end_time = time.time()
    print("Time taken: " + str(end_time - start_time))

    for key, value in found_solutions.items():
        print(key + ": " + value)

    print("Attempted " + str(attempts) + " problems:")
    print("Solved " + str(solves) + "/" + str(attempts) + ". in " + str(round(end_time - start_time)) + "s")


if RUN_TYPE == "LOCAL":
    task_path = training_path if not RUN_LOCAL_ON_EVAL else evaluation_path
    tasks = sorted(os.listdir(task_path))

    tryToSolveTrainingTasks(task_path, tasks)

TOO_HARD_PROBLEMS = {
    "31aa019c.json": "find the only dot with a color, delete everything else and surround that dot with a red border",
    "178fcbfb.json": "draw a vertical line on each red dot then draw a horizontal line on all green or blue lines",
    "1a07d186.json": "move each dot to touch a line of the same color. If no line, delete the dot.",
    "1caeab9d.json": "for each object, move to align with the blue object",
    "1e0a9b12.json": "for each object, move down as far as possible",
    "b8cdaf2b.json": "", # Find the highest object (no diagonals). From left side draw a -1, -1 line in SECONDARY_COLOR. From right side draw a (-1,+1) line in SECONDARY_COLOR
    "7df24a62.json": "", # Find the SECONDARY_COLOR object. Copy it and align by PRIMARY_COLOR for all matching attachment points, allowing rotation
    "99fa7670.json": "",  # forEachDot(orderByLowest(DOTS), Motion(drawLine(dot, "RIGHT"), drawLine(DOWN)))
    # TODO: Can I account for consistencies between input/output pairs (line long/skinny) to indicate a sequence problem?
    "e9afcf9a.json": "",  # horizontalSequence(forEveryOther(column: flipVertical(column)))
    "72ca375d.json": "findObjectWhere(XISLANDS, lambda object: object.hasHorizontalSymmetry)",
    "ecdecbb3.json": "", # forEachDot(DOTS, lambda dot: forEachObject(filterToLineOfSight(dot, LINES))), lambda line: doBoth(drawLineToObject(dot, line, colorOf(dot), includeEnds=true), drawBorderOutside(dot, colorOf(line))))
    "913fb3ed.json": "forEachDot(lambda dot: drawDotBorder(dot, fromMap(dot.color, [[GREEN, PURPLE], [LIGHTBLUE, YELLOW], [RED, BLUE]])))",
}

KNOWN_SOLUTIONS_MAP_TRAINING = {
    # TODO: Make this work by fixing the BIGGEST_BLOCK search
    "91714a58.json": "doBoth(clear(), putObject(BIGGEST_BLOCK_OBJECT))",

    "007bbfb7.json": "fractal()",
    "00d62c1b.json": "forEachBounds(LAKES, lambda bounds247: fillBoundsWithColor(bounds247, 4))",
    "025d127b.json": "forEachObject(lambda object: mutateObject())",
    "017c7c7b.json": "doBoth(renderSequence(SEQUENCE), replaceColor(1, 2))",
    "0520fde7.json": "doBoth(joinMultiUsingAnd(splitHorizontal()), replaceColor(1, 2))",
    "05f2a901.json": "moveObjectToTouchObject(LARGEST_MONO_OBJECT, SMALLEST_MONO_OBJECT)",
    "08ed6ac7.json": "applyColorSequence(sortSmallestFirst(ISLANDS), [YELLOW, GREEN, RED, BLUE], lambda object, color: fillBoundsWithColor(object, color))",
    "0b148d64.json": "cropToBounds(SECONDARY_BOUNDING_BOX)",
    "1cf80156.json": "cropToBounds(findSmallest(ISLANDS))",
    "2013d3e2.json": "grabPart(ALL_COLOR_BOUNDS_OBJECT, 1, 1)",
    "22168020.json": "forEachInputColor(lambda color: connectColinearPoints(filterDotsByColor(CDOTS, color), color))",
    "23b5c85d.json": "cropToObject(SMALLEST_MONO_OBJECT)",
    "253bf280.json": "connectColinearPoints(DOTS, GREEN)",
    "25ff71a9.json": "translateObject(LARGEST_MONO_OBJECT, 1, 0)",
    "28bf18c6.json": "repeatTile(LARGEST_MONO_OBJECT, 1, 2)",
    "2dc579da.json": "cropToObject(findUnique(getColoredObjects(PRIMARY_COLOR), lambda object: countColors(object)))",
    "32597951.json": "replaceColorInBounds(1, 3, getBoundingBoxOfColor(8))",
    "3618c87e.json": "forEachObject(getColoredObjects(BLUE), lambda object: translateObject(object, 2, 0))",
    "3aa6fb7a.json": "forEachBounds(ISLANDS, lambda bounds247: replaceColorInBounds(0, 1, bounds247))",
    "3af2c5a8.json": "kaleidoscope()",
    "3c9b0459.json": "rotate180()",
    "4258a5f9.json": "forEachBounds(ISLANDS, lambda bounds811: drawBorder(bounds811, 1))",
    "4347f46a.json": "forEachBounds(ISLANDS, lambda bounds247: fillBoundsWithColor(shrinkBounds(bounds247), 0))",
    "46442a0e.json": "joinFourTurns()",
    "4c4377d9.json": "doBoth(flipVertical(), mirrorVertical(0))",
    "50cb2852.json": "forEachBounds(ISLANDS, lambda bounds247: fillBoundsWithColor(shrinkBounds(bounds247), 8))",
    "5582e5ca.json": "fillBoundsWithColor(FULL_BOUNDS, PRIMARY_COLOR)",
    "56ff96f3.json": "forEachInputColor(lambda color: fillBoundsWithColor(getBoundingBoxOfColor(color), color))",
    "5c0a986e.json": "doBoth(drawLineFromObject(getColoredObject(RED), 1, 1), drawLineFromObject(getColoredObject(BLUE), -1, -1))",
    "60b61512.json": "forEachObject(XISLANDS, lambda object: replaceColorInBounds(0, ORANGE, object))",
    "6150a2bd.json": "rotate180()",
    "623ea044.json": "forEachDot(DOTS, lambda dot: doBoth(drawLine(dot, -1, -1, True), drawLine(dot, -1, 1, True)))",
    "62c24649.json": "kaleidoscope()",
    "6430c8c4.json": "invert(joinMultiUsingOr(splitVertical()), GREEN)",
    "67385a82.json": "forEachObject(filterOutDots(ISLANDS), lambda object: replaceColorInBounds(GREEN, LIGHTBLUE, object))",
    "67a3c6ac.json": "flipHorizontal()",
    "67e8384a.json": "kaleidoscope()",
    "68b16354.json": "flipVertical()",
    "694f12f3.json": "applyColorSequence(sortSmallestFirst(ISLANDS), [BLUE, RED], lambda object, color: fillBoundsWithColor(shrinkBounds(object), color))",
    "6d75e8bb.json": "replaceColorInBounds(0, 2, findSmallest(ISLANDS))",
    "6f8cd79b.json": "drawBorder(shrinkBounds(FULL_BOUNDS), LIGHTBLUE)",
    "6fa7a44f.json": "mirrorVertical(0)",
    # TODO: This broke after flipHorizontal stopped accepting parameters
    "7468f01a.json": "doBoth(cropToObject(findSmallest(ISLANDS)), flipHorizontal())",
    "74dd1130.json": "doBoth(flipVertical(), rotate90())",
    "7b6016b9.json": "doBoth(forEachObject(LAKES, lambda object: replaceColorInBounds(0, RED, object)), replaceColor(0, GREEN))",
    "7ddcd7ec.json": "forEachDot(DOTS, lambda dot: drawLineAwayFromObject(dot, BIGGEST_BLOCK_OBJECT))",
    "7fe24cdd.json": "joinFourTurns()",
    "810b9b61.json": "forEachBounds(LAKES, lambda bounds811: drawBorder(bounds811, 3))",
    "88a62173.json": "cropToObject(findUnique(XISLANDS, identity))",
    "8be77c9e.json": "mirrorVertical(0)",
    "8d510a79.json": "doBoth(forEachDot(filterDotsByColor(DOTS, RED), lambda dot: drawLineToObject(dot, LINE)), forEachDot(filterDotsByColor(DOTS, BLUE), lambda dot: drawLineAwayFromObject(dot, LINE)))",
    "9172f3a0.json": "scaleUp(3)",
    "928ad970.json": "drawBorder(shrinkBounds(shrinkBounds(SECONDARY_BOUNDING_BOX)), PRIMARY_COLOR)",
    "9565186b.json": "doBoth(fillBoundsWithColor(FULL_BOUNDS, GRAY), putObject(LARGEST_MONO_OBJECT))",
    "963e52fc.json": "repeatTile(FULL_BOUNDS, 1, 2)",
    "99b1bc43.json": "doBoth(joinMultiUsingXor(splitVertical()), replaceColor(PRIMARY_COLOR, GREEN))",
    "9dfd6313.json": "doBoth(flipVertical(), rotate90())",
    "9ecd008a.json": "doBoth(rotate90(), cropToObject(findSmallest(LAKES)))",
    "a416b8f3.json": "repeatTile(FULL_BOUNDS, 1, 2)",
    "a48eeaf7.json": "forEachDot(DOTS, lambda dot: moveDotToObject(dot, LARGEST_MONO_OBJECT))",
    "a5313dff.json": "forEachBounds(LAKES, lambda bounds247: replaceColorInBounds(0, 1, bounds247))",
    "a68b268e.json": "doBoth(joinMultiUsingOr(splitHorizontal()), joinMultiUsingOr(splitVertical()))",
    "a79310a0.json": "doBoth(translateObject(LARGEST_MONO_OBJECT, 1, 0), replaceColor(PRIMARY_COLOR, RED))",
    "a87f7484.json": "cropToObject(findUnique(slice(3, 3, 0), bwIdentity))",
    "aabf363d.json": "doBoth(replaceColor(SECONDARY_COLOR, 0), replaceColor(PRIMARY_COLOR, SECONDARY_COLOR))",
    "b1948b0a.json": "replaceColor(6, 2)",
    "bb43febb.json": "forEachBounds(ISLANDS, lambda bounds247: fillBoundsWithColor(shrinkBounds(bounds247), 2))",
    "be94b721.json": "cropToBounds(PRIMARY_BOUNDING_BOX)",
    "c59eb873.json": "scaleUp(2)",
    "c8f0f002.json": "replaceColorInBounds(7, 5, PRIMARY_BOUNDING_BOX)",
    "ce22a75a.json": "forEachBounds(ISLANDS, lambda bounds247: fillBoundsWithColor(growBounds(bounds247), 1))",
    "d037b0a7.json": "forEachDot(CDOTS, lambda dot812: drawLine(dot812, 1, 0))",
    "d10ecb37.json": "cropToObject(LEARNED_OUTPUT_OBJECT)",
    "d9fac9be.json": "doBoth(cropToObject(LEARNED_OUTPUT_OBJECT), replaceColor(0, SECONDARY_COLOR))",
    "dbc1a6ce.json": "connectColinearPoints(DOTS, LIGHTBLUE)",
    "dc0a314f.json": "doBoth(rotate180(), cropToBounds(getBoundingBoxOfColor(3)))",
    "dc1df850.json": "forEachDot(filterDotsByColor(DOTS, RED), lambda dot: drawDotBorder(dot, BLUE))",
    "ded97339.json": "connectColinearPoints(CDOTS, 8)",
    "e9614598.json": "drawObjectCenteredOnPoint(LEARNED_OUTPUT_OBJECT, getCenterPointOf(getBoundingBoxOfColor(PRIMARY_COLOR), False))",
    "e98196ab.json": "joinMultiUsingOr(splitVertical())",
    "ea32f347.json": "applyColorSequence(sortSmallestFirst(ISLANDS), [RED, YELLOW, BLUE], lambda object, color: fillBoundsWithColor(object, color))",
    "eb281b96.json": "doBoth(mirrorVertical(1), mirrorVertical(1))",
    "ed36ccf7.json": "doBoth(rotate90(), rotate180())",
    "f25fbde4.json": "doBoth(cropToBounds(LARGEST_MONO_OBJECT), scaleUp(2))",
    "fcb5c309.json": "doBoth(cropToBounds(findLargest(MONO_OBJECTS)), replaceColor(PRIMARY_COLOR, SECONDARY_COLOR))",
    "ff805c23.json": "doBoth(rotate180(), cropToBounds(getBoundingBoxOfColor(1)))",
}
print("Solution dictionary contains " + str(len(KNOWN_SOLUTIONS_MAP_TRAINING)) + "/400 solutions");

# Technically discoverable solutions that are too deep (ex: 5 layers) to be found in a complex search
TOO_DEEP_SOLUTIONS = {
    "3de23699.json": "doBoth(cropToBounds(shrinkBounds(getBoundingBoxOfColor(SECONDARY_COLOR))), replaceColor(PRIMARY_COLOR, SECONDARY_COLOR))",
}


def loadTrainingGrid(filename):
    global GRID
    with open(training_path / filename, 'r') as f:
        task = json.load(f)

    GRID = task["train"][0]["input"]

    conclusions = setTaskwideValues(task)

    setPrecomputedValues(getPrecomputedValues(GRID))

    return GRID


def runExecutionVerification():
    attempts = 0
    solved = 0
    for file, solution in KNOWN_SOLUTIONS_MAP_TRAINING.items():
        if FOCUS_TASK is not None and file != FOCUS_TASK:
            continue

        task_file = str(training_path / file)

        with open(task_file, 'r') as f:
            task = json.load(f)

        print("Trying " + file)

        passed = trySolveTask(task, useSolution=solution, debug=True)
        attempts += 1
        solved += 1 if passed else 0
    print(str(solved) + "/" + str(attempts))


def runSearchVerification():
    task_path = training_path  # evaluation_path
    tasks = list(KNOWN_SOLUTIONS_MAP_TRAINING.keys())
    solutions = list(KNOWN_SOLUTIONS_MAP_TRAINING.values())

    print("Verifying that answers are found for all known solutions")
    tryToSolveTrainingTasks(task_path, tasks, solutions)


if RUN_TYPE == "VERIFICATION":
    if VERIFICATION_TYPE == "EXECUTE" or VERIFICATION_TYPE == "BOTH":
        runExecutionVerification()
    if VERIFICATION_TYPE == "SEARCH" or VERIFICATION_TYPE == "BOTH":
        runSearchVerification()

In [0]:
def plot_one(task, ax, i, train_or_test, input_or_output):
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9', '#FF4136', '#2ECC40', '#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)

    input_matrix = task[train_or_test][i][input_or_output]
    ax.imshow(input_matrix, cmap=cmap, norm=norm)
    ax.grid(True, which='both', color='lightgrey', linewidth=0.5)
    ax.set_yticks([x - 0.5 for x in range(1 + len(input_matrix))])
    ax.set_xticks([x - 0.5 for x in range(1 + len(input_matrix[0]))])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title(train_or_test + ' ' + input_or_output)


def plot_full_task(task):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """
    num_train = len(task['train'])
    fig, axs = plt.subplots(2, num_train, figsize=(3 * num_train, 3 * 2))
    for i in range(num_train):
        plot_one(task, axs[0, i], i, 'train', 'input')
        plot_one(task, axs[1, i], i, 'train', 'output')
    plt.tight_layout()
    plt.show()

    num_test = len(task['test'])
    fig, axs = plt.subplots(2, num_test, figsize=(3 * num_test, 3 * 2))
    if num_test == 1:
        plot_one(task, axs[0], 0, 'test', 'input')
        plot_one(task, axs[1], 0, 'test', 'output')
    else:
        for i in range(num_test):
            plot_one(task, axs[0, i], i, 'test', 'input')
            plot_one(task, axs[1, i], i, 'test', 'output')
    plt.tight_layout()
    plt.show()

# Reasoning and Problem Exploration
# The next section will try to go beyond the limits of a static-generative approach by analyzing input/output pairs for relevant differences, which could theoretically be used to more directly find a solution within the program generation DSL

In [0]:

def runReasoningV1():
    task_path = training_path  # evaluation_path
    tasks = sorted(os.listdir(task_path))

    task_count = 15
    for filename in tasks:
        task_count -= 1
        if task_count < 0:
            break

        with open(str(task_path / filename), 'r') as f:
            task = json.load(f)

        print("Reasoning about " + filename)
        reasonAboutTask(task, True)

    return True


if RUN_TYPE == "REASONING":
    runReasoningV1()

# Invariant Object Identification
So there's probably some nice ways to do this with NN or other bullshit, but I want to build a series of identifiers for objects that are invariant across color and/or rotation, which are matchable properties
#
#
testGrid = [[1, 0, 1, 1], [1, 0, 1, 1], [0, 1, 1, 1], [0, 0, 1, 1]]
result = findBiggestBoxes(testGrid)
#
#
if RUN_TYPE == "EXPLORATION":
    runExploration()